In [ ]:
from IPython.core.display import display, HTML

In [ ]:
HTML('''<script>code_show=true;function code_toggle()
{if (code_show){$('div.input').hide();} 
else{$('div.input').show();}code_show = !code_show}$( document ).ready(code_toggle);
</script><em>The raw code in this jupyter notebook is hidden by default for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</,!a></em>.''')

In [ ]:
# install this to use some extra jupyter features (like content, this really helps navigate)
#! jupyter contrib nbextension install --system
#! pip install jupyter_contrib_nbextensions

In [ ]:
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
HTML("""
<style>

div.cell { /* Tunes the space between cells */
margin-top:5px;
margin-bottom:10px;
}

div.text_cell_render { /* Customize text cells */
font-family: 'Times New Roman';
font-size:20px;
line-height:1.4em;
padding-left:3em;
padding-right:3em;
}
</style>
""")

<font size="5">Import various general modules</font>

In [ ]:
import sys
import myPlots
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = (12, 7)
plt.rc('text', usetex=False)
from scipy import constants
from scipy.integrate import simps
import warnings
warnings.filterwarnings("ignore")
import itertools as it
import pickle

<font size="5">Assign a saving folder (where all the data and plots will be saved)</font>

In [ ]:
saveFolder='./results/skySimulation/'

<font size="5">Set Offline path</font>

In [ ]:
offlinePath='/home/max/auger/soft/offline/install/'

Interactive or inline plotting (by default the plots are inline).

In [ ]:
# for interactive plots (zoom, etc) use:
# %matplotlib notebook #uncomment this
# to turn it off use:
# %matplotlib inline #uncomment this

In [ ]:
# If it does not work try some of these:
# import mpld3
# mpld3.enable_notebook()
# mpld3.disable_notebook()
# %matplotlib tk  
# %matplotlib qt #etc. - GUI windows show the figure externally to the notebook with the given interactive backend

# Antenna pattern
<p style='text-align: justify;'>
The class <i>getAntennaPatternQuantities</i> reads the Offline XML file with the Antenna pattern. Further, it allows you to transform the antenna pattern's Offline azimuth convention 0$^\circ$,360$^\circ$ (with 0$^\circ$ at East and North at +90$^\circ$) to azimuth convention -180$^\circ$,180$^\circ$ (With East at +90$^\circ$ and West at -90$^\circ$) and from zenith to altitude. This convention is what is internal used by healpy. Also, you can interpolate the antenna pattern. See the examples. <br>
***
!!! Note that the NS antenna pattern in Offline is align to NS sensitivity. During the run Offline properly rotates the antenna pattern.
See RModels_RD.xml and check the part with $<$OrientationAzimuth$>$.
That is why the convention transformation in the class <i>getAntennaPatternQuantities</i> is different for the EW and NS orientation.
 </p>

In [ ]:
# modules need by this class
from collections import OrderedDict
import os
from lxml import etree as ET
from io import StringIO
from scipy.interpolate import interp1d

class getAntennaPatternQuantities():
    def __init__(self,antennaPath):
        self._frequencyXML, phiXML, thetaXML, MeanTransferXML, self._theta_ampXML, self._theta_phaseXML,\
        self._phi_ampXML, self._phi_phaseXML = self._read_rp(antennaPath)
        self._phi = list(OrderedDict.fromkeys(phiXML))
        self._theta = list(OrderedDict.fromkeys(thetaXML))
        self._templateDF = self._DFtemplateCreator(self._theta,self._phi,yName='phi')
    def _DFtemplateCreator(self,xs,ys,yName=None):
        colList = [str(x) for x in xs]
        nan_init = np.empty((len(xs),len(ys),))
        nan_init[:]=np.nan
        templateDF = pd.DataFrame(nan_init.T, columns = colList)
        templateDF.insert(0,yName,ys)
        templateDF = templateDF.set_index(yName)
        return templateDF
    def _interpolate(self,df,newAxisValues, axis,kind='linear',
        copy=True, bounds_error=None, fill_value=np.nan, assume_sorted=False):
        if axis == 0:
            InterpFunc = interp1d(df.index.values.astype(float),df.values,kind=kind,axis=0,
            copy=copy, bounds_error=bounds_error, fill_value=fill_value, assume_sorted=assume_sorted)
            templateDF = self._DFtemplateCreator(df.columns.values.astype(float),newAxisValues, yName='phi')
        elif axis == -1:
            InterpFunc = interp1d(df.columns.values.astype(float),df.values,kind=kind,axis=-1,
            copy=copy, bounds_error=bounds_error, fill_value=fill_value, assume_sorted=assume_sorted)
            templateDF = self._DFtemplateCreator(newAxisValues,df.index.values.astype(float), yName='phi')
        newValues = InterpFunc(newAxisValues)
        templateDF.iloc[:,:] = newValues
        return templateDF
    def get(self, frequency=45, quantity='', newPhi=None, newTheta=None,changeConvention=False,orientation=False,
        kind='linear',copy=True, bounds_error=None, fill_value=np.nan, assume_sorted=False):
        #print('Getting quantity: '+quantity+' at frequency: '+str(frequency))
        findex = np.where(self._frequencyXML==frequency)[0][0]
        temporaryDF = self._templateDF.copy(deep=True)
        if quantity != 'absHeight':
            exec('temporaryDF.iloc[:,:] = self._'+quantity+'XML[findex].reshape(len(self._phi),len(self._theta))')
        elif quantity == 'absHeight':
            absHeight = np.sqrt( (1/2)* (self._theta_ampXML[findex].reshape(len(self._phi),len(self._theta))**2 + 
            self._phi_ampXML[findex].reshape(len(self._phi),len(self._theta))**2)   )
            temporaryDF.iloc[:,:] = absHeight
        if changeConvention == True:
            temporaryDF = self._changeAzimuthDefinitionAndZenith2Altitude(temporaryDF,orientation=orientation)
        # if interpolate
        if newPhi is not None:
            temporaryDF = self._interpolate(temporaryDF, newPhi, axis=0, kind=kind,
            copy=copy, bounds_error=bounds_error, fill_value=fill_value, assume_sorted=assume_sorted)
        if newTheta is not None:
            temporaryDF = self._interpolate(temporaryDF, newTheta, axis=-1, kind=kind,
            copy=copy, bounds_error=bounds_error, fill_value=fill_value, assume_sorted=assume_sorted)
        return temporaryDF
    def _changeAzimuthDefinitionAndZenith2Altitude(self, df,orientation=False):
        #def changeAzimuthDefinitionAndZenith2Altitude(temporaryDF):
        # drop last coll since it is just copy of the first row to make things periodical
        df = df.drop(index=360.0)
        # change the azimuth order from counterclockwise to clockwise by -1 and rotate it by 90 deg to align North to 0
        newIndex = np.asarray(df.index)
        if orientation == 'EW':
            newIndex=-(newIndex-90)
        elif orientation == 'NS':
            newIndex=-(newIndex-180)
        else:
            print('Set orientation of the antenna!')
        # fix values outside the -180 to 180 interval
        newIndex[np.where(newIndex<-180)] = newIndex[np.where(newIndex< -180)]+360
        # set new index and sort by it
        df=df.set_index(newIndex)
        df=df.sort_index()
        # find which periodical row is missing and add it
        try:
            lastRow = df.loc[-180.0]
            lastRow.name=180.0
        except:
            lastRow = df.loc[180.0]
            lastRow.name=-180.0
        df=df.append(lastRow)
        df=df.sort_index()
        # now change zenith to altitude
        cols=df.columns.tolist()
        df=df[cols[::-1]]
        df.columns=cols
        return df
    def getPhiAndThetaGrid(self):
        return np.meshgrid(self._phi, self._theta)
    def _read_rp(self, path): # credits to KIT for this funcion If the original author find this, send me your name :) !!!
        def get_component(root, comp):
            component = []
            for x in root.findall(comp):
                component.append(string_to_float_array(x.text))
            return np.array(component)
        def string_to_float_array(string, det=" "):# credits to KIT for this funcion
            return np.array([float(x) for x in string.split(det) if x.strip()])
        fopen = open(path, "r")
        fstring = fopen.read()
        # to create proper root
        fstring = "<rp>" + fstring + "</rp>"
        # get parser and tree
        parser = ET.XMLParser(remove_blank_text=True, remove_comments=True)
        tree = ET.parse(StringIO(fstring), parser)
        # get root object
        rp_root = tree.getroot()
        frequency = string_to_float_array(rp_root.find("frequency").text)
        phi = string_to_float_array(rp_root.find("phi").text)
        theta = string_to_float_array(rp_root.find("theta").text)
        MeanTransfer = string_to_float_array(rp_root.find("MeanTransfer").text)
        theta_amp = get_component(rp_root, "EAHTheta_amp")
        theta_phase = get_component(rp_root, "EAHTheta_phase")
        phi_amp = get_component(rp_root, "EAHPhi_amp")
        phi_phase = get_component(rp_root, "EAHPhi_phase")
        for x in [theta_phase, phi_amp, phi_phase]:
            if theta_amp.shape != x.shape:
                sys.exit("Mismatch component")
        if len(frequency) != len(MeanTransfer):
            sys.exit("Mismatch MeanTransfer")
        if len(frequency) != theta_amp.shape[0] or len(phi) != theta_amp.shape[1] or len(theta) != theta_amp.shape[1]:
            print(len(frequency), len(phi), len(theta), theta_amp.shape)
            sys.exit("Mismatch directions")
        return frequency, phi, theta, MeanTransfer, theta_amp, theta_phase, phi_amp, phi_phase

Set path to the antenna pattern XML files.

In [ ]:
#antennaPathEW = offlinePath+'/share/auger-offline/config/SALLA_RD_NEC2_with-WCD-SSD_EW.xml'
#antennaPathNS = offlinePath+'/share/auger-offline/config/SALLA_RD_NEC2_with-WCD-SSD_NS.xml'
antennaPathEW = '/home/max/auger/soft/offline/Framework/RDetector/AntennaModels/SALLA/SALLA_RD_NEC2_with-WCD-SSD_EW_GND-6-Very_Poor.xml'
antennaPathNS = '/home/max/auger/soft/offline/Framework/RDetector/AntennaModels/SALLA/SALLA_RD_NEC2_with-WCD-SSD_NS_GND-6-Very_Poor.xml'

Create antenna class for EW and NS orientation.

In [ ]:
antennaEW = getAntennaPatternQuantities(antennaPathEW)
antennaNS = getAntennaPatternQuantities(antennaPathNS)

## Examples

### Plot of antenna pattern quantity Q at frequency F

In [ ]:
# select frequency
frequency=45

# choose quantity
quantity = 'phi_amp'
# quantity = 'theta_amp'
# quantity = 'absHeight'

# select EW orientation
orientation = 'EW'
antenna=antennaEW
Amp = antenna.get(quantity=quantity,frequency=frequency)

close=False
# save=False
save=saveFolder+orientation+"_"+quantity+'_'+str(frequency)+'Mhz'

P, T = np.meshgrid(Amp.index.values.astype(float),Amp.columns.values.astype(float))
myPlots.plot3dnewV3(P, T, Amp.values.T,slices=True, ymax=91.5, xMajorLocator=30, yMajorLocator=10, figureWidth=12, figureHeight=8,
                    xlabel='phi$\degree$',ylabel='theta$\degree$',cbarLabel=quantity+' [m]',
                    mainTitle=orientation+" orientation at frequency "+str(frequency)+' [MHz]',
                    save=save,close=close)
    
# select NS orientation
orientation = 'NS'
antenna=antennaNS
Amp = antenna.get(quantity=quantity,frequency=frequency)
save=False
save=saveFolder+orientation+"_"+quantity+'_'+str(frequency)+'Mhz'

P, T = np.meshgrid(Amp.index.values.astype(float),Amp.columns.values.astype(float))
myPlots.plot3dnewV3(P, T, Amp.values.T,slices=True, ymax=91.5, xMajorLocator=30, yMajorLocator=10, figureWidth=12, figureHeight=8,
                    xlabel='phi$\degree$',ylabel='theta$\degree$',cbarLabel=quantity+' [m]',
                    mainTitle=orientation+" orientation at frequency "+str(frequency)+' [MHz]',
                    save=save,close=close)

### Plot of antenna pattern quantity Q at frequency F with interpolated phi and theta coordinates

In [ ]:
incTheta = 0.1
incPhi = 0.1
newTheta = np.arange(0,90+incTheta,incTheta)
newPhi= np.arange(0,360+incPhi,incPhi)

# select frequency
frequency=45

# choose quantity
quantity = 'phi_amp'
# quantity = 'theta_amp'
# quantity = 'absHeight'

# select EW orientation
orientation = 'EW'
antenna=antennaEW

# select NS orientation
# orientation = 'NS'
# antenna=antennaNS

close=False
# save=False
save=saveFolder+orientation+"_"+quantity+'_'+str(frequency)+'Mhz'+'_thetaInterp_cubic'

# kind = linear, cubic etc. see scipy interp1d for options
Amp = antenna.get(quantity=quantity, frequency=frequency, newTheta=newTheta, newPhi=newPhi, kind='cubic', changeConvention=False, orientation='EW')

P, T = np.meshgrid(Amp.index.values.astype(float),Amp.columns.values.astype(float))
myPlots.plot3dnewV3(P, T, Amp.values.T,slices=True, ymax=91.5, xMajorLocator=30, yMajorLocator=10, figureWidth=12, figureHeight=8,
                    xlabel='phi$\degree$',ylabel='theta$\degree$',cbarLabel=quantity+' [m]',
                    mainTitle=orientation+" orientation at frequency "+str(frequency)+' [MHz]',
                    save=save, close=close)

### Plot of antenna pattern quantity Q at frequency F with the changed convention
Change from the Offline azimuth convention to -180$^{\circ}$ and 180$^{\circ}$ azimuth with North at 0$^{\circ}$  and East at 90$^{\circ}$, West at -90$^{\circ}$ and from zenith to altitude. Or in other words, from Offline convention to healpy internal standard.

In [ ]:
# select frequency
frequency=45

# choose quantity
# quantity = 'phi_amp'
# quantity = 'theta_amp'
quantity = 'absHeight'

# select EW orientation
orientation = 'EW'
antenna=antennaEW

# select NS orientation
# orientation = 'NS'
# antenna=antennaNS

close=False
# save=False
save=saveFolder+orientation+"_"+quantity+'_'+str(frequency)+'Mhz'

# changeConvention=True,orientation='EW', EW and NS has to be treated differently since both are in Offline align to North
Amp = antenna.get(quantity=quantity, frequency=frequency, changeConvention=True, orientation='EW')


P, T = np.meshgrid(Amp.index.values.astype(float),Amp.columns.values.astype(float))
myPlots.plot3dnewV3(P, T, Amp.values.T, slices=True, ymax=91.5, xMajorLocator=30, yMajorLocator=10, figureWidth=12, figureHeight=8,
                    xlabel='phi$\degree$',ylabel='theta$\degree$',cbarLabel=quantity+' [m]',
                    mainTitle=orientation+" orientation at frequency "+str(frequency)+' [MHz]',
                    save=save,close=close)

### Plot of antenna pattern quantity Q at frequency F with the changed convention with interpolated values
This is the most usefull for the real work. We need to change the Offline convention to healpy convention with properly rotated NS antenna with a custom angle spacing.

In [ ]:
incTheta = 0.1
incPhi = 0.1
newTheta = np.arange(0,90+incTheta,incTheta)
newPhi= np.arange(-180,180+incPhi,incPhi)

# select frequency
frequency=45

# choose quantity
# quantity = 'phi_amp'
# quantity = 'theta_amp'
quantity = 'absHeight'

# select EW orientation
orientation = 'EW'
antenna=antennaEW

close=False
# save=False
save=saveFolder+orientation+"_"+quantity+'_'+str(frequency)+'Mhz'+'_thetaInterp_cubic'

# changeConvention=True,orientation='EW', EW and NS has to be treated differently since both are in Offline align to North
Amp = antenna.get(quantity=quantity, frequency=frequency, changeConvention=True, orientation=orientation, newTheta=newTheta, newPhi=newPhi,kind='cubic')

P, T = np.meshgrid(Amp.index.values.astype(float),Amp.columns.values.astype(float))
myPlots.plot3dnewV3(P, T, Amp.values.T,slices=True, ymax=91.5, xMajorLocator=30, yMajorLocator=10, figureWidth=12, figureHeight=8,
                    xlabel='phi$\degree$',ylabel='theta$\degree$',cbarLabel=quantity+' [m]',
                    mainTitle=orientation+" orientation at frequency "+str(frequency)+' [MHz]',
                    save=save,close=close)

# select NS orientation
orientation = 'NS'
antenna=antennaNS

# save=False
save=saveFolder+orientation+"_"+quantity+'_'+str(frequency)+'Mhz'+'_thetaInterp_cubic'

Amp = antenna.get(quantity=quantity, frequency=frequency, changeConvention=True, orientation=orientation, newTheta=newTheta, newPhi=newPhi, kind='cubic')

P, T = np.meshgrid(Amp.index.values.astype(float),Amp.columns.values.astype(float))
myPlots.plot3dnewV3(P, T, Amp.values.T,slices=True, ymax=91.5, xMajorLocator=30, yMajorLocator=10, figureWidth=12, figureHeight=8,
                    xlabel='phi$\degree$',ylabel='theta$\degree$',cbarLabel=quantity+' [m]',
                    mainTitle=orientation+" orientation at frequency "+str(frequency)+' [MHz]',
                    save=save,close=close)

# Radio sky temperature map simulations

<font size="5">Import necessery modules</font>

In [ ]:
import healpy

In [ ]:
# healpy as the manipulation tool
import healpy as hp
# the upgraded newvisufunc is not yet in the official release
# current it is a pull request https://github.com/healpy/healpy/pull/648
from healpy import newvisufunc

In [ ]:
# import pyGDSM module
sys.path.append("/workdata/LFSS")
from pygdsm import GlobalSkyModel2016, GlobalSkyModel, LowFrequencySkyModel, HaslamSkyModel

Import Polisensky's LFmap as a class  (you need to run the script LFmap_healpyFitsConvertorAndGenerator.py inside the LFmap software folder to generate and convert the fits format to healpy fits format).<br>
Note please, that Polisensky's LFmap, these are by default generated in Celestial coordinated.

<font size="5">Set path to the LFmap folder:</font>

In [ ]:
LFmapPath='/home/max/auger/soft/LFmap_1.0/'

In [ ]:
class LFmap():
    def __init__(self, path):
        self._path=path
    def generate(self,frequency):
        return hp.read_map(self._path+"LFmap_"+str(frequency)+"_healpy.fits",verbose=True)

Set path to GMOSS folder:

In [ ]:
GMOSS_path='/home/max/auger/soft/new_sky_models/GMOSS/'

In [ ]:
class GMOSS():
    def __init__(self, path):
        self._path=path
    def generate(self,frequency):
        return hp.read_map(self._path+'GMOSS_'+str(frequency).replace('.','_')+'_healpy.fits',verbose=True)

Set path to SSM folder:

In [ ]:
SSM_path='/home/max/auger/soft/new_sky_models/SSM/'

In [ ]:
class SSM():
    def __init__(self, path):
        self._path=path
    def generate(self,frequency):
        return hp.read_map(self._path+'SSM_'+str(frequency).replace('.','_')+'_healpy.fits',verbose=True)

Set path to ULSA folder (constant index):

In [ ]:
ULSA_constant_path='/home/max/auger/soft/new_sky_models/ULSA/constant_index/'

In [ ]:
class ULSA_c():
    def __init__(self, path):
        self._path=path
    def generate(self,frequency):
        return hp.read_map(self._path+'ULSA_'+str(frequency).replace('.','_')+'_healpy.fits',verbose=True)

Set path to ULSA folder (constant index, absorption free):

In [ ]:
ULSA_constant_absfree_path='/home/max/auger/soft/new_sky_models/ULSA/constant_index/absorption_free/'

In [ ]:
class ULSA_c_af():
    def __init__(self, path):
        self._path=path
    def generate(self,frequency):
        return hp.read_map(self._path+'ULSA_'+str(frequency).replace('.','_')+'_healpy.fits',verbose=True)

Set path to ULSA folder (freq dependent index):

In [ ]:
ULSA_freq_path='/home/max/auger/soft/new_sky_models/ULSA/freq_dependent_index/'

In [ ]:
class ULSA_f():
    def __init__(self, path):
        self._path=path
    def generate(self,frequency):
        return hp.read_map(self._path+'ULSA_'+str(frequency).replace('.','_')+'_healpy.fits',verbose=True)

Set path to ULSA folder (direction dependent index):

In [ ]:
ULSA_dir_path='/home/max/auger/soft/new_sky_models/ULSA/direction_dependent_index/'

In [ ]:
class ULSA_d():
    def __init__(self, path):
        self._path=path
    def generate(self,frequency):
        return hp.read_map(self._path+'ULSA_'+str(frequency).replace('.','_')+'_healpy.fits',verbose=True)

Set path to Remazeilles folder:

In [ ]:
Remazeilles_dir_path='/home/max/auger/soft/new_sky_models/Remazeilles/'

In [ ]:
class Remazeilles():
    def __init__(self, path):
        self._path=path
    def generate(self,frequency):
        return hp.read_map(self._path+'Remazeilles_'+str(frequency).replace('.','_')+'_healpy.fits',verbose=True)

Set path to Refs folder:

In [ ]:
Refs_dir_path='/home/max/auger/soft/galactic_calibration/sky_maps/reference_maps/'

In [ ]:
class Refs():
    def __init__(self, path):
        self._path=path
    def generate(self,frequency):
        return hp.read_map(self._path+str(frequency).replace('.','_')+'_MHz.fits',verbose=True)

Create the sky temperature map classes for both references.

In [ ]:
# for Polisensky's LFmap
g_LFmap = LFmap(LFmapPath+"/healpyFits/")
g_GMOSS = GMOSS(GMOSS_path+'/healpyFits/')
g_SSM = SSM(SSM_path+'/healpyFits/')
g_ULSA_c = ULSA_c(ULSA_constant_path+'/healpyFits/')
g_ULSA_f = ULSA_f(ULSA_freq_path+'/healpyFits/')
g_ULSA_c_af = ULSA_c_af(ULSA_constant_absfree_path+'/healpyFits/')
g_ULSA_d = ULSA_c_af(ULSA_dir_path+'/healpyFits/')
g_Remazeilles = Remazeilles(Remazeilles_dir_path+'/healpyFits/')
g_Refs = Refs(Refs_dir_path)
# for lf map from pygdsem
g_LFSS = LowFrequencySkyModel(freq_unit='MHz')
g_GSM = GlobalSkyModel(freq_unit='MHz')
g_GSM16 = GlobalSkyModel2016(freq_unit='MHz')
g_Haslam = HaslamSkyModel(freq_unit='MHz')

## Examples
For more examples check the notebook dedicated for this (skyMapExamples)

First generate maps at some frequency (for example 45 MHz). Please write the frequency in the format with the decimal. The class for LFmap is searching for the correct map by use of the filename.

In [ ]:
freq = 50.0

map_LFSS = np.log10(g_LFSS.generate(freq))
map_LFSS_title = "LFSS"
map_LFmap = np.log10(g_LFmap.generate(freq))
map_LFmap_title = "LFmap"
map_GSM = np.log10(g_GSM.generate(freq))
map_GSM_title = "GSM"
map_GSM16 = np.log10(g_GSM16.generate(freq))
map_GSM16_title = "GSM16"
map_GMOSS = np.log10(g_GMOSS.generate(freq))
map_GMOSS_title = "GMOSS"
map_SSM = np.log10(g_SSM.generate(freq))
map_SSM_title = "SSM"
map_ULSA_c = np.log10(g_ULSA_c.generate(freq))
map_ULSA_c_title = "ULSA (constant index)"
map_ULSA_f = np.log10(g_ULSA_f.generate(freq))
map_ULSA_f_title = "ULSA (frequency dependent index)"
map_ULSA_d = np.log10(g_ULSA_d.generate(freq))
map_ULSA_d_title = "ULSA (direction dependent index)"
map_ULSA_c_af = np.log10(g_ULSA_c_af.generate(freq))
map_ULSA_c_af_title = "ULSA (constant index, absorption free)"
map_Haslam = np.log10(g_Haslam.generate(freq))
map_Haslam_title = "Haslam"
#map_Remazeilles = np.log10(g_Remazeilles.generate(freq))
#map_Remazeilles_title = "Remazeilles"
map_Refs = g_Refs.generate(36.528)
map_Refs_title = "Refs"

###  Maps in Galactic coordinates
Also you can check other possible projections, like cart, hammer etc. Check skyMapExample notebook for this.

In [ ]:
from importlib import reload
reload(hp.newvisufunc)
import pygdsm

In [ ]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

'''
# mollweide projections
newvisufunc.projview(map_LFSS, coord=['G'],graticule=True, graticule_labels=True,
                    unit=r'$T/K$', #xlabel='longitude',ylabel='latitude',
                    cbticks=[3.5,4.0,4.5,5.0], cbticklabels=[r'$10^{3.5}$',r'$10^4$',r'$10^{4.5}$',r'$10^5$'],
                    cb_orientation='vertical',min=3.5,max=5,latitude_grid_spacing=30,
                    projection_type="mollweide",lcolor='white') #,title=map_LFSS_title
plt.savefig(saveFolder+'Example_map_LFSM.pdf', facecolor='white', bbox_inches='tight')

'''
newvisufunc.projview(map_LFmap, coord=['C','G'],graticule=True, graticule_labels=True,
                    unit=r'$T/K$', #xlabel='l',ylabel='b',
                    cbticks=[3.5,4.0,4.5,5.0], cbticklabels=[r'$10^{3.5}$',r'$10^4$',r'$10^{4.5}$',r'$10^5$'],
                    cb_orientation='vertical',min=3.5,max=5,latitude_grid_spacing=30,
                    projection_type="mollweide",lcolor='white') #,title=map_LFmap_title
plt.savefig(saveFolder+'Example_map_LFmap.pdf', facecolor='white', bbox_inches='tight')
'''
newvisufunc.projview(map_GSM, coord=['G'],graticule=True, graticule_labels=True,
                    unit=r'$T/K$', #xlabel='longitude',ylabel='latitude',
                    cbticks=[3.5,4.0,4.5,5.0], cbticklabels=[r'$10^{3.5}$',r'$10^4$',r'$10^{4.5}$',r'$10^5$'],
                    cb_orientation='vertical',min=3.5,max=5,latitude_grid_spacing=30,
                    projection_type="mollweide",lcolor='white') #,title=map_LFSS_title
plt.savefig(saveFolder+'Example_map_GSM.pdf', facecolor='white', bbox_inches='tight')

newvisufunc.projview(map_GSM16, coord=['G'],graticule=True, graticule_labels=True,
                    unit=r'$T/K$', #xlabel='longitude',ylabel='latitude',
                    cbticks=[3.5,4.0,4.5,5.0], cbticklabels=[r'$10^{3.5}$',r'$10^4$',r'$10^{4.5}$',r'$10^5$'],
                    cb_orientation='vertical',min=3.5,max=5,latitude_grid_spacing=30,
                    projection_type="mollweide",lcolor='white') #,title=map_LFSS_title
plt.savefig(saveFolder+'Example_map_GSM16.pdf', facecolor='white', bbox_inches='tight')

newvisufunc.projview(map_GMOSS, coord=['G'],graticule=True, graticule_labels=True,
                    unit=r'$T/K$', #xlabel='longitude',ylabel='latitude',
                    cbticks=[3.5,4.0,4.5,5.0], cbticklabels=[r'$10^{3.5}$',r'$10^4$',r'$10^{4.5}$',r'$10^5$'],
                    cb_orientation='vertical',min=3.5,max=5,latitude_grid_spacing=30,
                    projection_type="mollweide",lcolor='white') #,title=map_LFSS_title
plt.savefig(saveFolder+'Example_map_GMOSS.pdf', facecolor='white', bbox_inches='tight')

newvisufunc.projview(map_SSM, coord=['G'],graticule=True, graticule_labels=True,
                    unit=r'$T/K$', #xlabel='longitude',ylabel='latitude',
                    cbticks=[3.5,4.0,4.5,5.0], cbticklabels=[r'$10^{3.5}$',r'$10^4$',r'$10^{4.5}$',r'$10^5$'],
                    cb_orientation='vertical',min=3.5,max=5,latitude_grid_spacing=30,
                    projection_type="mollweide",lcolor='white') #,title=map_LFSS_title
plt.savefig(saveFolder+'Example_map_SSM.pdf', facecolor='white', bbox_inches='tight')

newvisufunc.projview(map_ULSA_c, coord=['G'],graticule=True, graticule_labels=True,
                    unit=r'$T/K$', #xlabel='longitude',ylabel='latitude',
                    cbticks=[3.5,4.0,4.5,5.0], cbticklabels=[r'$10^{3.5}$',r'$10^4$',r'$10^{4.5}$',r'$10^5$'],
                    cb_orientation='vertical',min=3.5,max=5,latitude_grid_spacing=30,
                    projection_type="mollweide",lcolor='white') #,title=map_LFSS_title
plt.savefig(saveFolder+'Example_map_ULSA_c.pdf', facecolor='white', bbox_inches='tight')

newvisufunc.projview(map_ULSA_f, coord=['G'],graticule=True, graticule_labels=True,
                    unit=r'$T/K$', #xlabel='longitude',ylabel='latitude',
                    cbticks=[3.5,4.0,4.5,5.0], cbticklabels=[r'$10^{3.5}$',r'$10^4$',r'$10^{4.5}$',r'$10^5$'],
                    cb_orientation='vertical',min=3.5,max=5,latitude_grid_spacing=30,
                    projection_type="mollweide",lcolor='white') #,title=map_LFSS_title
plt.savefig(saveFolder+'Example_map_ULSA_f.pdf', facecolor='white', bbox_inches='tight')

newvisufunc.projview(map_ULSA_d, coord=['G'],graticule=True, graticule_labels=True,
                    unit=r'$T/K$', #xlabel='longitude',ylabel='latitude',
                    cbticks=[3.5,4.0,4.5,5.0], cbticklabels=[r'$10^{3.5}$',r'$10^4$',r'$10^{4.5}$',r'$10^5$'],
                    cb_orientation='vertical',min=3.5,max=5,latitude_grid_spacing=30,
                    projection_type="mollweide",lcolor='white') #,title=map_LFSS_title
plt.savefig(saveFolder+'Example_map_ULSA_d.pdf', facecolor='white', bbox_inches='tight')

newvisufunc.projview(map_ULSA_c_af, coord=['G'],graticule=True, graticule_labels=True,
                    unit=r'$T/K$', #xlabel='longitude',ylabel='latitude',
                    cbticks=[3.5,4.0,4.5,5.0], cbticklabels=[r'$10^{3.5}$',r'$10^4$',r'$10^{4.5}$',r'$10^5$'],
                    cb_orientation='vertical',min=3.5,max=5,latitude_grid_spacing=30,
                    projection_type="mollweide",lcolor='white') #,title=map_LFSS_title
plt.savefig(saveFolder+'Example_map_ULSA_c_af.pdf', facecolor='white', bbox_inches='tight')

newvisufunc.projview(map_Haslam, coord=['G'],graticule=True, graticule_labels=True,
                    unit=r'$T/K$', #xlabel='longitude',ylabel='latitude',
                    cbticks=[3.5,4.0,4.5,5.0], cbticklabels=[r'$10^{3.5}$',r'$10^4$',r'$10^{4.5}$',r'$10^5$'],
                    cb_orientation='vertical',min=3.5,max=5,latitude_grid_spacing=30,
                    projection_type="mollweide",lcolor='white') #,title=map_LFSS_title
plt.savefig(saveFolder+'Example_map_Haslam.pdf', facecolor='white', bbox_inches='tight')
'''
'''
newvisufunc.projview(map_Remazeilles, coord=['G'],graticule=True, graticule_labels=True,
                    unit=r'$T/K$', #xlabel='longitude',ylabel='latitude',
                    cbticks=[3.5,4.0,4.5,5.0], cbticklabels=[r'$10^{3.5}$',r'$10^4$',r'$10^{4.5}$',r'$10^5$'],
                    cb_orientation='vertical',min=0.5,max=3,latitude_grid_spacing=30,
                    projection_type="mollweide",lcolor='white') #,title=map_LFSS_title
plt.savefig(saveFolder+'Example_map_Remazeilles.pdf', facecolor='white', bbox_inches='tight')
'''
newvisufunc.projview(map_Refs, coord=['G','C'],graticule=True, graticule_labels=True,
                    unit=r'$T/K$', #xlabel='l',ylabel='b',
                    #cbticks=[3.5,4.0,4.5,5.0], cbticklabels=[r'$10^{3.5}$',r'$10^4$',r'$10^{4.5}$',r'$10^5$'],
                    #cbticks=[0,2.0e4,4.0e4,6.0e4,8.0e4], cbticklabels=['0','20000','40000','60000','80000'],
                    cb_orientation='vertical',latitude_grid_spacing=30,#  min=-1.5e4,max=9e4, cmap='magma',
                    projection_type="mollweide",lcolor='white') #,title=map_LFmap_title
plt.savefig(saveFolder+'Example_map_Refs.pdf', facecolor='white', bbox_inches='tight')

###  Transformation to the Local coordinates system

Rotate the maps to PAO local coordinates at the some particular LST time.

In [ ]:
# PAO altitude is -35.206667 degs
LSTtime=18
altitude = -35.206667

Transform the coordinates and make the plots.

In [ ]:
# Local coordinates at LST time "LSTtime" at altitude "altitude"
rotAngles=[(180+(LSTtime*15))%360,-(altitude-90)]
newvisufunc.projview(map_LFSS,rot=rotAngles, coord=['G','C'],graticule=True, graticule_labels=True,
                    unit='Temperature log[K]',xlabel='Azimuth',ylabel='Altitude',
                    cb_orientation='vertical',min=3.5,max=5,latitude_grid_spacing=30,
                    projection_type="cart",title='LFSS - Local coordinates',lcolor='white')

newvisufunc.projview(map_LFmap, rot=rotAngles, coord=['C'],graticule=True, graticule_labels=True,
                    unit='Temperature log[K]',xlabel='Azimuth',ylabel='Altitude',
                    cb_orientation='vertical',min=3.5,max=5,latitude_grid_spacing=30,
                    projection_type="cart",title="Polisensky's LFmap - Local coordinates",lcolor='white')
#The default is to measure azimuth East from North
# 0 is north 90 is east 180 is souht, 270 is west

### Data dump and DataFrame creation
An example of how to dump the data from the maps. This is very important because later we need to convolute the sky temperature maps with the antenna pattern. Always check the dump data by plotting them (for example with function myPlots.plot3d()). <br>
The data dump will return three variables: <br>
longitude, latitude, grid_map <br>
where the longtitude goes from -pi to pi (-180 to 180 in degs) and the latitude goes from -pi/2 to pi/2 (-90$^\circ$ to 90$^\circ$) <br>
Longitude and latitude are 1D arrays, to convert them to 2D arrays for the plot use np.meshgrid(longitude,  latitude)

In [ ]:
# return only data
# [longitude,  latitude, grid_map]
# longitude,  latitude are 1D arrays to convert them to 2D arrays for the plot use np.meshgrid(longitude,  latitude)
# longtitude goes from -pi to pi (-180 to 180 in degs)
# latitude goes from -pi/2 to pi/2 (-90 to 90 in degs)
# generate maps at 45 MHz
# do not logarithm this here!
map_LFSS = g_LFSS.generate(45.0)
map_LFSS_title = "LFSS"
map_LFmap = g_LFmap.generate(45.0)
map_LFmap_title = "LFmap"

rotAngles=[(180+(LSTtime*15))%360,-(altitude-90)]
longitude_LFSS,  latitude_LFSS, grid_map_LFSS = newvisufunc.projview(map_LFSS, rot=rotAngles, coord=['G','C'],
                                                      return_only_data=True)
longitude_LFmap,  latitude_LFmap, grid_map_LFmap = newvisufunc.projview(map_LFmap, rot=rotAngles, coord=['C'],
                                                      return_only_data=True)

Check the shapes of the dumped variables.

In [ ]:
print(longitude_LFmap.shape)
print(latitude_LFmap.shape)
print(grid_map_LFmap.shape)

Further, it is good to convert the dumped data to a proper Pandas DataFrame. The function <i>convert2SkyDF</i> will create a Pandas DataFrame from the dumped data with columns as frequencies and rows (index) as local sidereal times.

In [ ]:
# this function helps to convert the data dump from newvisufunc.projview() to DF
def convert2SkyDF(data_dump,full=False):
    def DFtemplateCreator(xs,ys,yName=None):
        colList = [str(x) for x in xs]
        nan_init = np.empty((len(xs),len(ys),))
        nan_init[:]=np.nan
        templateDF = pd.DataFrame(nan_init.T, columns = colList)
        templateDF.insert(0,yName,ys)
        templateDF = templateDF.set_index(yName)
        return templateDF
    longitude,latitude,grid_map = data_dump
    latitudeDeg = np.rad2deg(latitude)
    longitudeDeg = np.rad2deg(longitude)
    skyMapDF = DFtemplateCreator(latitudeDeg,longitudeDeg,yName=None)
    skyMapDF.iloc[:,:] = grid_map.T
    if full is False:
        skyMapDF = skyMapDF.iloc[:,np.where(skyMapDF.columns.astype(float) >=0)[0].tolist()]
    skyMapDF=skyMapDF.sort_index()
    return skyMapDF

Define catalogs of all available radio sky interpolation models.

In [ ]:
# for map from LFmap, these are by default generated in Celestial coordinated
index_upper, index_lower = -2.46, -2.62
model = {"LFmap": LFmap("/home/max/auger/soft/LFmap_1.0/healpyFits/"), "LFSM": LowFrequencySkyModel(freq_unit='MHz'), "GSM": GlobalSkyModel(freq_unit='MHz'), "GSM16": GlobalSkyModel2016(freq_unit='MHz'), "HaslamL": HaslamSkyModel(freq_unit='MHz', spectral_index=index_lower), "HaslamU": HaslamSkyModel(freq_unit='MHz', spectral_index=index_upper), "GMOSS": GMOSS("/home/max/auger/soft/new_sky_models/GMOSS/healpyFits/"), "SSM": SSM("/home/max/auger/soft/new_sky_models/SSM/healpyFits/"), "ULSA": ULSA_c("/home/max/auger/soft/new_sky_models/ULSA/constant_index/healpyFits/"), "ULSA_f": ULSA_f("/home/max/auger/soft/new_sky_models/ULSA/freq_dependent_index/healpyFits/"), "ULSA_d": ULSA_d("/home/max/auger/soft/new_sky_models/ULSA/direction_dependent_index/healpyFits/"), "ULSA_c_af": ULSA_c_af("/home/max/auger/soft/new_sky_models/ULSA/constant_index/absorption_free/healpyFits/")}
model_title = {"LFmap": "LFmap", "LFSM": "LFSM", "GSM": "GSM2008", "GSM16": "GSM2016", "HaslamL": "Haslam", "HaslamU": "Haslam", "Cane": "Cane", "Cane2": "TKY", 'GMOSS': 'GMOSS', 'SSM': 'SSM', 'ULSA': 'ULSA', 'ULSA_f': 'ULSA (frequency dependent index)', 'ULSA_d': 'ULSA', 'ULSA_c_af': 'ULSA (constant index, absorption free)'}
g_model_coord = {"LFmap": ['C','G'], "LFSM": ['G'], "GSM": ['G'], "GSM16": ['G'], "HaslamL": ['G'], "HaslamU": ['G'], "GMOSS": ['G'], "SSM": ['G'], "ULSA": ['G'], "ULSA_f": ['G'], "ULSA_d": ['G'], "ULSA_c_af": ['G']}
c_model_coord = {"LFmap": ['C'], "LFSM": ['G','C'], "GSM": ['G','C'], "GSM16": ['G','C'], "HaslamL": ['G','C'], "HaslamU": ['G','C'], "GMOSS": ['G','C'], "SSM": ['G','C'], "ULSA": ['G','C'], "ULSA_f": ['G','C'], "ULSA_d": ['G','C'], "ULSA_c_af": ['G','C']}
e_model_coord = {"LFmap": ['C','E'], "LFSM": ['G','E'], "GSM": ['G','E'], "GSM16": ['G','E'], "HaslamL": ['G','E'], "HaslamU": ['G','E'], "GMOSS": ['G','E'], "SSM": ['G','E'], "ULSA": ['G','E'], "ULSA_f": ['G','E'], "ULSA_d": ['G','E'], "ULSA_c_af": ['G','E']}
model_scale = {"LFmap": 256, "LFSM": 16, "GSM": 4, "GSM16": 1, "HaslamL": 16, "HaslamU": 16}

#choose sky models from: LFmap, LFSM, GSM, GSM16, Haslam
key1 = "GSM16"
key2 = "LFmap"

g_1 = model[key1]
g_2 = model[key2]
g_coord_1 = g_model_coord[key1]
g_coord_2 = g_model_coord[key2]
c_coord_1 = c_model_coord[key1]
c_coord_2 = c_model_coord[key2]
e_coord_1 = e_model_coord[key1]
e_coord_2 = e_model_coord[key2]
model_scale_1 = model_scale[key1]
model_scale_2 = model_scale[key2]

First generate the data.

In [ ]:
# generate sky maps
frequency = 30.0
map_1 = g_1.generate(frequency)
map_1_title = model_title[key1]
map_2 = g_2.generate(frequency)
map_2_title = model_title[key2]

# Colormap ranges for log scales
Tmin = 3.0
Tmax = 5.0

Dump the data.

In [ ]:
# dump maps in Galactic coordinate system
dumpG_1 = newvisufunc.projview(map_1,coord=g_coord_1, return_only_data=True)
dumpG_2 = newvisufunc.projview(map_2,coord=g_coord_2, return_only_data=True)

In [ ]:
print(dumpG_1[2])

Then convert the data dump to DataFrame.

In [ ]:
# then convert the data dump to DataFrame
# set full=True to have full Y axis
GskyMapDF_1=convert2SkyDF(dumpG_1,full=True)
GskyMapDF_2=convert2SkyDF(dumpG_2,full=True)

Check the created DataFrame.

In [ ]:
GskyMapDF_1

In [ ]:
GskyMapDF_2

Plot the data from the DataFrame and compare the maps by dividing one by the other one.

In [ ]:
# mollweide projections
newvisufunc.projview(np.log10(map_1), coord=g_coord_1,graticule=True, graticule_labels=True,
                    unit='Temperature log[K]',xlabel='l',ylabel='b',
                    cb_orientation='vertical',min=Tmin,max=Tmax,latitude_grid_spacing=30,
                    projection_type="mollweide",title=map_1_title+" - Galactic coordinates",lcolor='white')

newvisufunc.projview(np.log10(map_2), coord=g_coord_2,graticule=True, graticule_labels=True,
                    unit='Temperature log[K]',xlabel='l',ylabel='b',
                    cb_orientation='vertical',min=Tmin,max=Tmax,latitude_grid_spacing=30,
                    projection_type="mollweide",title=map_2_title+" - Galactic coordinates",lcolor='white')

### Comparison of the 1st map with the 2nd map at Local Coordinates

In [ ]:
plt.rc('text', usetex=False)
#plt.rc('font', family='monospace')

P, T = np.meshgrid(GskyMapDF_1.index.values.astype(float), GskyMapDF_1.columns.values.astype(float))

save=False
save1= saveFolder+"GskyMapDF_"+map_1_title+'_'+str(frequency).replace('.','_')
save2= saveFolder+"GskyMapDF_"+map_2_title+'_'+str(frequency).replace('.','_')
save3= saveFolder+"Ratio_GskyMapDF_"+map_1_title+"-GskyMapDF_"+map_2_title+'_'+str(frequency).replace('.','_')
save4=saveFolder+"nonAbs_Difference_GskyMapDF_"+map_1_title+"-GskyMapDF_"+map_2_title+'_'+str(frequency).replace('.','_')
save5=saveFolder+"Abs_Difference_GskyMapDF_"+map_1_title+"-GskyMapDF_"+map_2_title+'_'+str(frequency).replace('.','_')

# log here!
myPlots.plot3dnewV3(P, T, np.log10(GskyMapDF_1.values.T),
                    slices=True, ymax=90.5, xMajorLocator=30,yMajorLocator=30, figureWidth=12, figureHeight=8,
                    xlabel='l$\degree$',ylabel='b$\degree$',cbarLabel='Temperature log[K]',
                    mainTitle=map_1_title+" Sky temperature in Galactic coords at freq "+str(frequency)+' MHz',Cmin=Tmin,Cmax=Tmax,extend='both',
                    save=save1,close=close)

P, T = np.meshgrid(GskyMapDF_2.index.values.astype(float),GskyMapDF_2.columns.values.astype(float))
myPlots.plot3dnewV3(P, T, np.log10(GskyMapDF_2.values.T),
                    slices=True, ymax=90.5, xMajorLocator=30,yMajorLocator=30, figureWidth=12, figureHeight=8,
                    xlabel='l$\degree$',ylabel='b$\degree$',cbarLabel='Temperature log[K]',
                    mainTitle=map_2_title+" Sky temperature in Galactic coords at freq "+str(frequency)+' MHz',Cmin=Tmin,Cmax=Tmax,extend='both',
                    save=save2,close=close)

ratio=GskyMapDF_1.values.T/GskyMapDF_2.values.T
P, T = np.meshgrid(GskyMapDF_1.index.values.astype(float),GskyMapDF_1.columns.values.astype(float))
myPlots.plot3dnewV3(P, T, ratio,
                    slices=True, ymax=90.5, xMajorLocator=30,yMajorLocator=30, figureWidth=12, figureHeight=8,
                    xlabel='l$)*\degree$',ylabel='b$\degree$',cbarLabel=r'$\frac{\mathrm{%s}}{\mathrm{%s}}$'%(map_1_title, map_2_title),
                    mainTitle="Ratio of "+map_1_title+" and "+map_2_title+" sky temp. in Galactic coords at freq "+str(frequency)+' MHz',Cmin=0,Cmax=2,extend='max',
                    save=save3,close=close)

difference=np.sign(GskyMapDF_1.values.T-GskyMapDF_2.values.T)*np.log(abs(GskyMapDF_1.values.T-GskyMapDF_2.values.T))
P, T = np.meshgrid(GskyMapDF_1.index.values.astype(float),GskyMapDF_1.columns.values.astype(float))
myPlots.plot3dnewV4(P, T, difference,
                    slices=True, ymax=90.5, xMajorLocator=30,yMajorLocator=30,figureWidth=12,figureHeight=8, Cmap='coolwarm',
                    xlabel='l$\degree$',ylabel='b$\degree$',cbarLabel='log(T/K) \n '+r'${\mathrm{%s}}-{\mathrm{%s}}$'%(map_1_title, map_2_title),
                    mainTitle="Difference of "+map_1_title+" and "+map_2_title+" sky temp. at freq "+str(frequency)+' MHz',extend='max',
                    save=save4,close=close)

Abs_difference=np.log(abs(GskyMapDF_1.values.T-GskyMapDF_2.values.T))
P, T = np.meshgrid(GskyMapDF_1.index.values.astype(float),GskyMapDF_1.columns.values.astype(float))
myPlots.plot3dnewV3(P, T, Abs_difference,
                    slices=True, ymax=90.5, xMajorLocator=30,yMajorLocator=30,figureWidth=12,figureHeight=8,
                    xlabel='l$\degree$',ylabel='b$\degree$',cbarLabel='log(T/K) \n '+r'${\mathrm{%s}}-{\mathrm{%s}}$'%(map_1_title, map_2_title),
                    mainTitle="Abs. Difference of "+map_1_title+" and "+map_2_title+" sky temp. at freq "+str(frequency)+' MHz',extend='max',
                    save=save5,close=close)

Dump the maps transfomed to a local coordinates.

In [ ]:
# PAO altitude is -35.206667 degs
LSTtime=18
altitude = -35.206667

In [ ]:
rotAngles=[(180+(LSTtime*15))%360,-(altitude-90)]
dump_1 = newvisufunc.projview(map_1,rot=rotAngles, coord=c_coord_1,
                                                      return_only_data=True, xsize=newPhi.size)
dump_2 = newvisufunc.projview(map_2,rot=rotAngles, coord=c_coord_2,
                                                      return_only_data=True, xsize=newPhi.size)

In [ ]:
# add a sun to the local sky at any point (with right ascension+declination) with a uniform temperature and with an angular size in units of sunradii
def addSun_Local(SkyMap, RA=180., decl=45., Temp=45000., radius=0.25, cov=False):
    p = np.deg2rad(SkyMap.index.values.astype(float))
    t = np.deg2rad(SkyMap.columns.values.astype(float))
    T, P = np.meshgrid(t, p)
    s = np.vstack(([T.T],[(P+np.pi).T])).T
    #s2 = np.arccos(np.sin(np.rad2deg(s[...,0]))*np.sin(np.rad2deg(decl))+np.cos(np.rad2deg(s[...,0]))*np.cos(np.rad2deg(decl))*np.cos(np.rad2deg(s[...,1]-RA)))
    s2 = np.arccos(np.sin(s[...,0])*np.sin(decl)+np.cos(s[...,0])*np.cos(decl)*np.cos(s[...,1]-RA))
    if cov:
        val = np.where(s2 < np.deg2rad(radius), 100., 0.)
    else:
        val = np.where(s2 < np.deg2rad(radius), Temp, SkyMap.values)
    print(sum(sum(np.where(s2 < np.deg2rad(radius),1,0)))," pixels covered by the sun")
    #map_with_sun = pd.DataFrame(val, p, t)
    map_with_sun = pd.DataFrame(val, np.rad2deg(p), np.rad2deg(t))
    return map_with_sun

Then convert the data dumps to the DataFrames.

In [ ]:
LskyMapDF_1 = convert2SkyDF(dump_1,full=False)
LskyMapDF_2 = convert2SkyDF(dump_2,full=False)
LskyMapDF_Sun = addSun_Local(LskyMapDF_1, RA=180., decl=45., Temp=1e6, radius=2)
LskyMapDF_Sun = addSun_Local(LskyMapDF_Sun, RA=180., decl=.7, Temp=1e6, radius=2)
LskyMapDF_Sun = addSun_Local(LskyMapDF_Sun, RA=180., decl=.6, Temp=1e6, radius=2)
LskyMapDF_Sun = addSun_Local(LskyMapDF_Sun, RA=180., decl=.5, Temp=1e6, radius=2)
LskyMapDF_Sun = addSun_Local(LskyMapDF_Sun, RA=180., decl=.4, Temp=1e6, radius=2)

Plot the data from the DataFrame and compare the maps by dividing one by the other one.

In [ ]:
P, T = np.meshgrid(LskyMapDF_1.index.values.astype(float), LskyMapDF_1.columns.values.astype(float))

save=False
save1=saveFolder+"LskyMapDF_"+map_1_title+'_'+str(frequency).replace('.','_')
save2=saveFolder+"LskyMapDF_"+map_2_title+'_'+str(frequency).replace('.','_')
save3=saveFolder+"Ratio_LskyMapDF_"+map_1_title+"-LskyMapDF_"+map_2_title+'_'+str(frequency).replace('.','_')

# log here!
myPlots.plot3dnewV3(P, T, np.log10(LskyMapDF_Sun.values.T),
                    slices=True, ymax=90.5, xMajorLocator=30,yMajorLocator=15, figureWidth=12, figureHeight=8,
                    #xlabel='azimuth$\degree$',ylabel='altitude$\degree$', cbarLabel='Temperature log[K]', extend='both',
                    mainTitle=map_1_title+" Sky temperature in Local coords at freq "+str(frequency)+' MHz at '+str(LSTtime)+" LST",Cmin=Tmin,Cmax=Tmax,
                    save=save1, close=close)
'''
myPlots.plot3dnewV3(P, T, np.log10(LskyMapDF_2.values.T),
                    slices=True, ymax=90.5, xMajorLocator=30,yMajorLocator=15, figureWidth=12, figureHeight=8,
                    xlabel='azimuth$\degree$',ylabel='altitude$\degree$', cbarLabel='Temperature log[K]', extend='both',
                    mainTitle=map_2_title+" Sky temperature in Local coords at freq "+str(frequency)+' MHz at '+str(LSTtime)+" LST",Cmin=Tmin,Cmax=Tmax,
                    save=save2, close=close)

ratio=LskyMapDF_1.values.T/LskyMapDF_2.values.T
myPlots.plot3dnewV3(P, T, ratio,
                    slices=True, ymax=90.5, xMajorLocator=30,yMajorLocator=15, figureWidth=12, figureHeight=8,
                    xlabel='azimuth$\degree$', ylabel='altitude$\degree$', cbarLabel=r'$\frac{\mathrm{%s}}{\mathrm{%s}}$'%(map_1_title, map_2_title),
                    mainTitle="Ratio of "+map_1_title+" and "+map_2_title+" sky temp. in Local coords at freq "+str(frequency)+' MHz at '+str(LSTtime)+" LST",
                    Cmin=0, Cmax=2, extend='max', save=save3,close=close)
'''

## Plot Maps in Equatorial Coordinates

In [ ]:
LSTtime = 0.
rotAngles=[((LSTtime*15))%360,0]
dump_1 = newvisufunc.projview(map_1, coord=e_coord_1, return_only_data=True, xsize=newPhi.size)
dump_2 = newvisufunc.projview(map_2, coord=e_coord_2, return_only_data=True, xsize=newPhi.size)
#dump_1 = newvisufunc.projview(map_1, rot=rotAngles, coord=e_coord_1, return_only_data=True, xsize=newPhi.size)
#dump_2 = newvisufunc.projview(map_2, rot=rotAngles, coord=e_coord_2, return_only_data=True, xsize=newPhi.size)

Then convert the data dumps to the DataFrames.

In [ ]:
EskyMapDF_1=convert2SkyDF(dump_1,full=True)
EskyMapDF_2=convert2SkyDF(dump_2,full=True)

In [ ]:
def markCoverage(SkyMap, decl_min=-90., decl_max=90.):
    p = np.deg2rad(SkyMap.index.values.astype(float))
    t = np.deg2rad(SkyMap.columns.values.astype(float))
    T, P = np.meshgrid(t, p)
    val = np.where(np.logical_and(T >= np.deg2rad(decl_min), T <= np.deg2rad(decl_max)), SkyMap.values, 0.)
    map_cut = pd.DataFrame(val, p, t)
    return map_cut

In [ ]:
EskyMapDF_1_coverage = markCoverage(EskyMapDF_1, -40, 90)
EskyMapDF_2_coverage = markCoverage(EskyMapDF_2, -40, 90)

In [ ]:
# add a sun to the sky at any point (with right ascension+declination) with a uniform temperature and with an angular size in units of sunradii
def addSun(SkyMap, RA=180., decl=0., Temp=45000., radius=0.25, cov=False):
    p = np.deg2rad(SkyMap.index.values.astype(float))
    t = np.deg2rad(SkyMap.columns.values.astype(float))
    T, P = np.meshgrid(t, p)
    s = np.vstack(([T.T],[(P+np.pi).T])).T
    #s2 = np.arccos(np.sin(np.rad2deg(s[...,0]))*np.sin(np.rad2deg(decl))+np.cos(np.rad2deg(s[...,0]))*np.cos(np.rad2deg(decl))*np.cos(np.rad2deg(s[...,1]-RA)))
    s2 = np.arccos(np.sin(s[...,0])*np.sin(decl)+np.cos(s[...,0])*np.cos(decl)*np.cos(s[...,1]-RA))
    if cov:
        val = np.where(s2 < np.deg2rad(radius), 100., 0.)
    else:
        val = np.where(s2 < np.deg2rad(radius), Temp, SkyMap.values)
    print(sum(sum(np.where(s2 < np.deg2rad(radius),1,0)))," pixels covered by the sun")
    #map_with_sun = pd.DataFrame(val, p, t)
    map_with_sun = pd.DataFrame(val, np.rad2deg(p), np.rad2deg(t))
    return map_with_sun

In [ ]:
EskyMapDF_1_sun = addSun(EskyMapDF_1, RA=3.*np.pi/2., Temp=1e6, radius=10.)
print(type(EskyMapDF_1_sun))
print(type(EskyMapDF_1))

Plot the data from the DataFrame and compare the maps by dividing one by the other one.

In [ ]:
plt.rc('text', usetex=False)

P, T = np.meshgrid(EskyMapDF_1.index.values.astype(float), EskyMapDF_1.columns.values.astype(float))

# Convert to Right Ascension (alpha)
P = -(P - 180.)

save=False
save1=saveFolder+"EskyMapDF_"+map_1_title+'_'+str(frequency).replace('.','_')
save1_sun=saveFolder+"EskyMapDF_"+map_1_title+'_'+str(frequency).replace('.','_')+'_sun'
save1_coverage=saveFolder+"EskyMapDF_"+map_1_title+'_'+str(frequency).replace('.','_')+'_coverage'
save2=saveFolder+"EskyMapDF_"+map_2_title+'_'+str(frequency).replace('.','_')
save2_coverage=saveFolder+"EskyMapDF_"+map_2_title+'_'+str(frequency).replace('.','_')+'_coverage'
save3=saveFolder+"Ratio_EskyMapDF_"+map_1_title+'_sun_'+str(frequency).replace('.','_')

# log here!
myPlots.plot3dnewV3(P, T, np.log10(EskyMapDF_1_sun.values.T),
                    slices=True, ymax=90.5, xMajorLocator=30,yMajorLocator=15, figureWidth=12, figureHeight=8,
                    xlabel='right ascension$\degree$',ylabel='declination$\degree$', cbarLabel='Temperature log[K]', extend='both',
                    mainTitle=map_1_title+" Sky temperature in Equatorial coords at freq "+str(frequency)+' MHz',Cmin=Tmin,Cmax=Tmax,
                    save=save1_sun, close=close)

myPlots.plot3dnewV3(P, T, np.log10(EskyMapDF_1.values.T),
                    slices=True, ymax=90.5, xMajorLocator=30,yMajorLocator=15, figureWidth=12, figureHeight=8,
                    xlabel='right ascension$\degree$',ylabel='declination$\degree$', cbarLabel='Temperature log[K]', extend='both',
                    mainTitle=map_1_title+" Sky temperature in Equatorial coords at freq "+str(frequency)+' MHz',Cmin=Tmin,Cmax=Tmax,
                    save=save1, close=close)
'''
myPlots.plot3dnewV3(P, T, np.log10(EskyMapDF_1.values.T),
                    slices=True, ymax=90.5, xMajorLocator=30,yMajorLocator=15, figureWidth=12, figureHeight=8, Blind_min=-90, Blind_max=-40,
                    xlabel='right ascension$\degree$',ylabel='declination$\degree$', cbarLabel='Temperature log[K]', extend='both',
                    mainTitle=map_1_title+" Sky temperature in Equatorial coords at freq "+str(frequency)+' MHz',Cmin=Tmin,Cmax=Tmax,
                    save=save1_coverage, close=close)

myPlots.plot3dnewV3(P, T, np.log10(EskyMapDF_2.values.T),
                    slices=True, ymax=90.5, xMajorLocator=30,yMajorLocator=15, figureWidth=12, figureHeight=8,
                    xlabel='right ascension$\degree$',ylabel='declination$\degree$', cbarLabel='Temperature log[K]', extend='both',
                    mainTitle=map_2_title+" Sky temperature in Equatorial coords at freq "+str(frequency)+' MHz',Cmin=Tmin,Cmax=Tmax,
                    save=save2, close=close)

myPlots.plot3dnewV3(P, T, np.log10(EskyMapDF_2.values.T),
                    slices=True, ymax=90.5, xMajorLocator=30,yMajorLocator=15, figureWidth=12, figureHeight=8, Blind_min=-90, Blind_max=-40,
                    xlabel='right ascension$\degree$',ylabel='declination$\degree$', cbarLabel='Temperature log[K]', extend='both',
                    mainTitle=map_2_title+" Sky temperature in Equatorial coords at freq "+str(frequency)+' MHz',Cmin=Tmin,Cmax=Tmax,
                    save=save2_coverage, close=close)
'''
ratio=EskyMapDF_1_sun.values.T/EskyMapDF_1_sun.values.T
myPlots.plot3dnewV3(P, T, ratio,
                    slices=True, ymax=90.5, xMajorLocator=30,yMajorLocator=15, figureWidth=12, figureHeight=8,
                    xlabel='right ascension$\degree$', ylabel='declination$\degree$', cbarLabel=r'$\frac{\mathrm{%s}}{\mathrm{%s}}$'%(map_1_title, map_2_title),
                    mainTitle="Ratio of "+map_1_title+" and "+map_2_title+" sky temp. in Equatorial coords at freq "+str(frequency)+' MHz',
                    Cmin=0, Cmax=2, extend='max', save=save3,close=close)

# Plot the sky coverage of the interpolation models

In [ ]:
refs = {"LFmap": np.array([[-28.,80.],[-90.,90.]]),
          "LFSM": np.array([[-28.,80.],[-90.,90.],[-6.,74.],[-40.,90.],[-40.,90.],[-40.,90.],[-40.,90.],[-40.,90.],[-90.,65.]]), 
          "GSM": np.array([[-28.,80.],[-90.,90.],[-6.,74.],[-90.,65.]]), 
          "GSM16": np.array([[-28.,80.],[-90.,90.],[-6.,74.],[-90.,65.],[-25.,25.],[-25.,25.]]),
          "GMOSS": np.array([[-90.,90.],[-90.,90.],[-90.,90.],[-90.,90.],[-90.,90.],[-90.,90.]]),
          "SSM": np.array([[-6.,74.],[-28.,80.],[-90.,65.],[-25.,25.],[-25.,25.],[-90.,90.]]),
          "ULSA_d": np.array([[-90.,90.],[-40.,90.],[-40.,90.],[-40.,90.],[-40.,90.],[-40.,90.],[-40.,90.],[-40.,90.],[-40.,90.],[-90.,65.]])}
cmap = matplotlib.cm.get_cmap('viridis')

In [ ]:
from importlib import reload
reload(hp.newvisufunc)

In [ ]:
plt.rc('text', usetex=True)

for key in refs:
    e_coord = e_model_coord[key]
    g_coord = g_model_coord[key]
    g = model[key]
    m = g.generate(30.)
    dump_e = newvisufunc.projview(m, coord=e_coord, return_only_data=True, xsize=newPhi.size)
    dump_g = newvisufunc.projview(m, coord=g_coord, return_only_data=True)
    EskyMapDF = convert2SkyDF(dump_e,full=True)
    GskyMapDF = convert2SkyDF(dump_g,full=True)
    p = np.deg2rad(EskyMapDF.index.values.astype(float))
    t = np.deg2rad(EskyMapDF.columns.values.astype(float))
    T, P = np.meshgrid(t, p)
    val = 0.
    for ref in refs[key]:
        val = np.where(np.logical_and(T >= np.deg2rad(ref[0]), T <= np.deg2rad(ref[1])), val+1., val)
    coverage_map = pd.DataFrame(val, np.rad2deg(p), np.rad2deg(t))
    EskyMapDF = coverage_map
    
    nside = 264
    npix = hp.nside2npix(nside)

    P, T = np.meshgrid(EskyMapDF.index.values.astype(float), EskyMapDF.columns.values.astype(float))

    indices = hp.ang2pix(nside, -1.*np.deg2rad(T)+np.pi/2., np.deg2rad(P))
    indices = indices
    #print(indices)

    hpxmap = np.zeros(npix, dtype=np.float)
    #print(EskyMapDF.values.T.shape)
    hpxmap[indices] = EskyMapDF.values.T
    #print(hpxmap)

    plt.figure()
    newvisufunc.projview(hpxmap, coord=['E','G'], graticule=True, graticule_labels=True, graticule_alpha=1.0, cmap='Blues',
                        unit='Number of ref.\ maps',xlabel='',ylabel='', min=0, max=max([len(refs[k]) for k in refs]),
                        cb_orientation='vertical',latitude_grid_spacing=30, longitude_grid_spacing = 60,
                        projection_type="mollweide",lcolor='black') #,title=model_title[key]+" - Galactic coordinates"
    plt.savefig(saveFolder+"Coverage_Map_scaled_"+model_title[key]+'.pdf', bbox_inches='tight')
    
'''
plt.figure()
newvisufunc.projview(hpxmap, coord=['E','G'], graticule=True, graticule_labels=True, cmap='Greens',
                    unit='Coverage in number of maps',xlabel='l',ylabel='b',
                    cb_orientation='vertical',latitude_grid_spacing=30, longitude_grid_spacing = 60,
                    projection_type="mollweide",title=model_title[key]+" - Galactic coordinates",lcolor='white')
plt.savefig(saveFolder+"Coverage_Map_"+model_title[key]+'.pdf', bbox_inches='tight')

plt.figure()
newvisufunc.projview(hpxmap, coord=['E'], graticule=True, graticule_labels=True, cmap='Greens',
                    unit='Coverage in number of maps',xlabel='l',ylabel='b', min=0, max=max([len(refs[k]) for k in refs]),
                    cb_orientation='vertical',latitude_grid_spacing=30, longitude_grid_spacing = 60,
                    projection_type="mollweide",title=model_title[key]+" - Equatorial coordinates",lcolor='white')
plt.savefig(saveFolder+"Coverage_Map_equatorial_scaled_"+model_title[key]+'.pdf', bbox_inches='tight')

plt.figure()
newvisufunc.projview(hpxmap, coord=['E'], graticule=True, graticule_labels=True, cmap='Greens',
                    unit='Coverage in number of maps',xlabel='l',ylabel='b',
                    cb_orientation='vertical',latitude_grid_spacing=30, longitude_grid_spacing = 60,
                    projection_type="mollweide",title=model_title[key]+" - Equatorial coordinates",lcolor='white')
plt.savefig(saveFolder+"Coverage_Map_equatorial_"+model_title[key]+'.pdf', bbox_inches='tight')
'''

In [ ]:
plt.rc('text', usetex=False)

P, T = np.meshgrid(EskyMapDF.index.values.astype(float), EskyMapDF.columns.values.astype(float))

# Convert to Right Ascension (alpha)
P = -(P - 180.)

save=False
save1=saveFolder+"Sky_Coverage_"+key

# log here!
myPlots.plot3dnewV3(P, T, EskyMapDF.values.T,
                    slices=True, ymax=90.5, xMajorLocator=30,yMajorLocator=15, figureWidth=12, figureHeight=8,
                    xlabel='right ascension$\degree$',ylabel='declination$\degree$', cbarLabel='Coverage in # of maps', extend='both',
                    mainTitle=model_title[key]+' Sky Coverage',#Cmin=0,Cmax=1,
                    save=save1, close=close)

# Exemplary ratio maps for all models with respect to the average

In [ ]:
reload(newvisufunc)

In [ ]:
plt.rc('text', usetex=True)

models = ["LFmap", "LFSM", "GSM", "GSM16", 'GMOSS', 'SSM', 'ULSA_d'] 

'''avg_g = model["LFmap"]
avg_m = avg_g.generate(50.)
avg_dump_g = newvisufunc.projview(avg_m, coord=g_model_coord["LFSM"], return_only_data=True)
avg_GskyMapDF = convert2SkyDF(avg_dump_g,full=True)
avg_p = np.deg2rad(avg_GskyMapDF.index.values.astype(float))
avg_t = np.deg2rad(avg_GskyMapDF.columns.values.astype(float))
avg_T, avg_P = np.meshgrid(avg_t, avg_p)
'''

avg_val = np.zeros((500,1000))

#print(avg_GskyMapDF.values.T)

for key in models:
    #e_coord = e_model_coord[key]
    g_coord = g_model_coord[key]
    g = model[key]
    m = g.generate(50.)
    if key in ["GSM", "GMOSS"]:
        m = m + 2.72548
    #dump_e = newvisufunc.projview(m, coord=e_coord, return_only_data=True, xsize=newPhi.size)
    dump_g = newvisufunc.projview(m, coord=g_coord, return_only_data=True)
    #EskyMapDF = convert2SkyDF(dump_e,full=True)
    GskyMapDF = convert2SkyDF(dump_g,full=True)
    avg_val = avg_val + GskyMapDF.values.T
    
for key in models:
    #e_coord = e_model_coord[key]
    g_coord = g_model_coord[key]
    g = model[key]
    m = g.generate(50.)
    if key in ["GSM", "GMOSS"]:
        m = m + 2.72548
    #dump_e = newvisufunc.projview(m, coord=e_coord, return_only_data=True, xsize=newPhi.size)
    dump_g = newvisufunc.projview(m, coord=g_coord, return_only_data=True)
    #EskyMapDF = convert2SkyDF(dump_e,full=True)
    GskyMapDF = convert2SkyDF(dump_g,full=True)
    p = np.deg2rad(GskyMapDF.index.values.astype(float))
    t = np.deg2rad(GskyMapDF.columns.values.astype(float))
    P, T = np.meshgrid(GskyMapDF.index.values.astype(float), GskyMapDF.columns.values.astype(float))

    '''
    #T, P = np.meshgrid(t, p)
    avg_val = np.add(avg_val, GskyMapDF.values.T)
    
    avg_map = pd.DataFrame(avg_val, np.rad2deg(t), np.rad2deg(p))
    #EskyMapDF = avg_map
    '''
    
    #print(avg_val)
    #print(GskyMapDF.values.T*4./avg_val)
    
    nside = 264
    npix = hp.nside2npix(nside)
    #T, P = np.meshgrid(EskyMapDF.index.values.astype(float), EskyMapDF.columns.values.astype(float))    
    indices = hp.ang2pix(nside, -1.*np.deg2rad(T)+np.pi/2., np.deg2rad(P))  # -1.*np.deg2rad(T)+np.pi/2.    
    hpxmap = np.zeros(npix, dtype=np.float)

    #hpxmap[indices] = np.sign(GskyMapDF.values.T*4.-avg_val) * np.log10(abs(GskyMapDF.values.T*4.-avg_val))
    hpxmap[indices] = GskyMapDF.values.T*len(models)/avg_val
                              
    P, T = np.meshgrid(GskyMapDF.index.values.astype(float),GskyMapDF.columns.values.astype(float))
    '''
    myPlots.plot3dnewV3(P, T, np.log10(GskyMapDF.values.T),
                    slices=True, ymax=90.5, xMajorLocator=30,yMajorLocator=30, figureWidth=12, figureHeight=8,
                    xlabel='',ylabel='', min=Tmin, max=Tmax,
                    mainTitle="Ratio", close=close) #,extend='max'
    '''
    cmin, cmax = 0.7, 1.3
    plt.figure()
    newvisufunc.projview(hpxmap, coord=['G'], graticule=True, graticule_labels=True, graticule_alpha=0.0, cmap='RdBu_r',
                        unit=r'$\frac{T_\mathrm{ %s }}{T_\mathrm{average}}$'%(key),xlabel='',ylabel='', min= cmin, max=cmax, cbticks=np.arange(cmin, cmax, 0.1), cbticklabels=[str(round(tick,1)) for tick in np.arange(cmin, cmax, 0.1)],
                        cb_orientation='vertical',latitude_grid_spacing=30, longitude_grid_spacing = 60,
                        projection_type="mollweide",lcolor='black') #,title=model_title[key]+" - Galactic coordinates"
    plt.savefig(saveFolder+"Ratio_Map_to_average_"+model_title[key]+'.pdf', bbox_inches='tight')



# Calculate average temperature of global & local sky at all frequencies 

Function to integrate over the sky for averaging.

In [ ]:
# integrate the folded sky map over the angles
def integrateOverAngles_cos(SkyMap, t_min=-90, t_max=90):
    p = np.deg2rad(SkyMap.index.values.astype(float))
    t = np.deg2rad(SkyMap.columns.values.astype(float))
    T, P = np.meshgrid(t, p)
    #print(t[0],t[-1])
    angePrefactor=np.cos(T[:,::-1])
    angePrefactor = np.where(np.logical_and(t >= np.deg2rad(t_min), t <= np.deg2rad(t_max)), angePrefactor, 0.)
    #angePrefactor=1
    integratedOverAngles=simps(simps(SkyMap*angePrefactor,t),p)
    return integratedOverAngles

In [ ]:
#saveFolder='./results/skySimulation/'
#saveFolder='./results/skySimulation/test/'
#saveFolder='./results/skySimulation/Coverage_test/'
#saveFolder='./results/skySimulation/Extended_Spectrum/'
#saveFolder='./results/skySimulation/Latitude_variation/'
#saveFolder='./results/skySimulation/ARENA_proceedings/'
saveFolder='./results/skySimulation/new_models/'

# for testing different LFmap parameters
#saveFolder='./results/skySimulation/LFmap_test/'

Calculate average temperatures with required numerical precision.

In [ ]:
average_T_G_temp, average_T_L_temp = {},{}

In [ ]:
# rotate them to Local coordinates at some particular LST time
altitude = -35.206667 # Pierre Auger Observatory
#altitude = 52.90889 # LOFAR core
# generate sky maps
frequency_range = np.linspace(30.0,408.0,379)
hours = np.arange(0,24,24) #1
average_T_G, average_T_L = {},{}

#for model_key in model:
for model_key in ['HaslamL','HaslamU']:
    g = model[model_key]
    map_title = model_title[model_key]
    T_G_temp,T_L_temp = [],[]
    for freq in frequency_range:
        map = g.generate(freq)
        # first generate the data dump
        dumpG = newvisufunc.projview(map,coord=g_model_coord[model_key], return_only_data=True)

        # then convert the data dump to DataFrame
        # set full=True to have full Y axis
        GskyMapDF=convert2SkyDF(dumpG,full=True)
        print(model_key, freq)
        T_G_temp.append(integrateOverAngles_cos(GskyMapDF, t_min=-90, t_max=90)/integrateOverAngles_cos(GskyMapDF/GskyMapDF, t_min=-90, t_max=90))
        print(T_G_temp[-1])
        T_hour = []
        for hour in hours:
            rotAngles=[(180+(hour*15))%360,-(altitude-90)]
            dump = newvisufunc.projview(map,rot=rotAngles, coord=c_model_coord[model_key], return_only_data=True,xsize=newPhi.size)
            LskyMapDF=convert2SkyDF(dump,full=False)
            T_hour.append(integrateOverAngles_cos(LskyMapDF)/integrateOverAngles_cos(LskyMapDF/LskyMapDF))
        T_L_temp.append(T_hour)
    average_T_G[model_key] = np.array(T_G_temp)
    average_T_L[model_key] = np.array(T_L_temp)

In [ ]:
with open("averageT_data.pickle", "wb") as fout:
    pickle.dump([frequency_range, hours, average_T_G, average_T_L], fout)

In [ ]:
with open("averageT_data.pickle", "rb") as fin:
    frequency_range, hours, average_T_G, average_T_L = pickle.load(fin)

In [ ]:
average_T_G

In [ ]:
average_T_L

In [ ]:
average_T_G_temp['LFmap'] = average_T_G['LFmap']
average_T_G_temp['GSM'] = average_T_G['GSM']
average_T_G_temp['GSM16'] = average_T_G['GSM16']
average_T_G_temp['LFSM'] = average_T_G['LFSM']
average_T_G_temp['GMOSS'] = average_T_G['GMOSS']
average_T_G_temp['SSM'] = average_T_G['SSM']
average_T_G_temp['ULSA'] = average_T_G['ULSA']
average_T_G_temp['ULSA_d'] = average_T_G['ULSA_d']
average_T_G_temp['HaslamL'] = average_T_G['HaslamL']
average_T_G_temp['HaslamU'] = average_T_G['HaslamU']

average_T_L_temp['LFmap'] = average_T_L['LFmap']
average_T_L_temp['GSM'] = average_T_L['GSM']
average_T_L_temp['GSM16'] = average_T_L['GSM16']
average_T_L_temp['LFSM'] = average_T_L['LFSM']
average_T_L_temp['GMOSS'] = average_T_L['GMOSS']
average_T_L_temp['SSM'] = average_T_L['SSM']
average_T_L_temp['ULSA'] = average_T_L['ULSA']
average_T_L_temp['ULSA_d'] = average_T_L['ULSA_d']
average_T_L_temp['HaslamL'] = average_T_L['HaslamL']
average_T_L_temp['HaslamU'] = average_T_L['HaslamU']

In [ ]:
average_T_G['LFmap'] = average_T_G_temp['LFmap']
average_T_G['GSM'] = average_T_G_temp['GSM']
average_T_G['GSM16'] = average_T_G_temp['GSM16']
average_T_G['LFSM'] = average_T_G_temp['LFSM']
average_T_G['GMOSS'] = average_T_G_temp['GMOSS']
average_T_G['SSM'] = average_T_G_temp['SSM']
average_T_G['ULSA'] = average_T_G_temp['ULSA']
average_T_G['ULSA_d'] = average_T_G_temp['ULSA_d']
#average_T_G['HaslamL'] = average_T_G_temp['HaslamL']
#average_T_G['HaslamU'] = average_T_G_temp['HaslamU']

average_T_L['LFmap'] = average_T_L_temp['LFmap']
average_T_L['GSM'] = average_T_L_temp['GSM']
average_T_L['GSM16'] = average_T_L_temp['GSM16']
average_T_L['LFSM'] = average_T_L_temp['LFSM']
average_T_L['GMOSS'] = average_T_L_temp['GMOSS']
average_T_L['SSM'] = average_T_L_temp['SSM']
average_T_L['ULSA'] = average_T_L_temp['ULSA']
average_T_L['ULSA_d'] = average_T_L_temp['ULSA_d']
#average_T_L['HaslamL'] = average_T_L_temp['HaslamL']
#average_T_L['HaslamU'] = average_T_L_temp['HaslamU']

Re-add the CMB component because it was subtracted in the model

In [ ]:
average_T_G['GSM'] = average_T_G['GSM'] + 2.72548
average_T_G['GMOSS'] = average_T_G['GMOSS'] + 2.72548

In [ ]:
ITU_data = np.genfromtxt('ITU_Recommendation_Galactic_Noise.txt', delimiter=';')
frequencies_ITU, T_ITU = ITU_data[ITU_data[:, 0].argsort()].T
frequencies_ITU *= 1e-6

Galactic noise brightness model by Cane(1979) as taken from Dulk(2001) with a scaling correction to account for plane-pole variation (here called "Cane"). A model for the brightness temperature from Tokarsky(2017) is included, which is derived from Canes model and findings from other authors.

In [ ]:
def Cane(nu):
    correction_factor = 1.3
    c = 3.0e8
    k = 1.38e-23
    Ieg = 1.06e-20
    Ig = correction_factor * 2.48e-20
    tau = 5.0 * nu**(-2.1)
    I = Ig * nu**(-0.52)*(1-np.exp(-tau))/tau + Ieg * nu**(-0.8) * np.exp(-tau)
    T = I*c**2 / (2*(nu*1e6)**2 * k)
    return T

def Cane2(nu):
    #Ig = 1.06e-20
    #tau = 5.0 * nu**(-2.1)
    #I = Ig * nu**(-0.52)*(1-np.exp(-tau))/tau + Ieg * nu**(-0.8) * np.exp(-tau)
    T = 3.78e5 * (10.0/nu)**2.56
    return T

frequency_range_param = np.linspace(30.0,100.0,71)
average_T_G["Cane"] = Cane(frequency_range_param)
average_T_G["Cane2"] = Cane2(frequency_range_param)

In [ ]:
model_list = ["LFmap", "GSM", "GSM16", "LFSM", 'GMOSS', 'SSM', 'ULSA_d']  # , 'ULSA_f'
model_list_param = ["Cane", "Cane2"]

for (m_1,m_2) in it.combinations(model_list,2):
    res = 2*(simps(average_T_G[m_1],frequency_range)-simps(average_T_G[m_2],frequency_range))/(simps(average_T_G[m_1],frequency_range)+simps(average_T_G[m_2],frequency_range))
    print('2*(',m_1,'-',m_2,')/(',m_1,'+',m_2,')')
    print('')
    print(round(res*100.,1),'%')
    print('')

In [ ]:
model_list = ["LFmap", "LFSM", "GSM", "GSM16", 'GMOSS', 'SSM', 'ULSA_d'] # , 'ULSA_f'
model_list_param = ["Cane", "Cane2"]

for (m_1,m_2) in it.combinations(model_list,2):
    res = simps((2.*(abs((average_T_G[m_1]-average_T_G[m_2])/(average_T_G[m_1]+average_T_G[m_2])))),frequency_range)/(frequency_range[-1]-frequency_range[0])
    print('2*(',m_1,'-',m_2,')/(',m_1,'+',m_2,')')
    print('')
    print(round(res*100.,1),'%')
    print('')

In [ ]:
'''
map = g_Refs.generate(159)
dumpG = newvisufunc.projview(map,coord=['C'], return_only_data=True)
GskyMapDF=convert2SkyDF(dumpG,full=True)
identity = (GskyMapDF/GskyMapDF).fillna(1.)
T_av = integrateOverAngles_cos(GskyMapDF, t_min=-90, t_max=60)/integrateOverAngles_cos(identity, t_min=-90, t_max=60)
print(integrateOverAngles_cos(identity, t_min=-90, t_max=60))
print('159', T_av)
    
for ref_freq in [36.528, 41.76, 46.992, 52.224, 57.456, 62.688, 67.92, 73.152]:
    map = g_Refs.generate(ref_freq)
    dumpG = newvisufunc.projview(map,coord=['G','C'], return_only_data=True)
    GskyMapDF=convert2SkyDF(dumpG,full=True)
    identity = (GskyMapDF/GskyMapDF).fillna(1.)
    T_av = integrateOverAngles_cos(GskyMapDF, t_min=-30, t_max=90)/integrateOverAngles_cos(identity, t_min=-30, t_max=90)
    print(integrateOverAngles_cos(identity, t_min=-30, t_max=90))
    print(ref_freq, T_av)
'''

In [ ]:
rel_dif, rel_dif_withoutLFSM = [],[]
for (m_1,m_2) in it.combinations(model_list,2):
    rel_dif.append(2.*abs((average_T_G[m_1]-average_T_G[m_2])/(average_T_G[m_1]+average_T_G[m_2])))
    if not "LFSM" in [m_1,m_2]:
        rel_dif_withoutLFSM.append(rel_dif[-1])

LoA = np.maximum.reduce(rel_dif)
LoA_withoutLFSM = np.maximum.reduce(rel_dif_withoutLFSM) 

Plot the data.

In [ ]:
model_title['GSM'] = 'GSM (2008)'
model_title['GSM16'] = 'GSM (2016)'

In [ ]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['figure.figsize'] = (12, 6)

Haslam_label = "Haslam \n\huge{"+str(index_lower)+r" $< \beta <$ "+str(index_upper)+"}"
model_list = ["LFmap", "GSM", "GSM16", "LFSM", 'GMOSS', 'SSM', 'ULSA_d'] # , 'ULSA_f', 'ULSA_d'
colors = {"LFmap": "tab:blue", "LFSM": "tab:orange", "GSM": "tab:green", "GSM16": "tab:red", "Haslam": "grey", "Cane": "k", "Cane2": "m", 'GMOSS': 'tab:olive', 'SSM': 'lime', 'ULSA_d': 'tab:cyan'}  # , 'ULSA_f': 'tab:olive'
linestyles = {"LFmap": "solid", "LFSM": "solid", "GSM": "solid", "GSM16": "solid", "Haslam": "solid", "Cane": "dotted", "Cane2": "dotted", 'GMOSS': 'solid', 'SSM': 'solid', 'ULSA_d': 'solid'}
label_ITU = "ITU"
color_ITU = "magenta"

#Plot average sky temperature
plt.figure()
for model_key in model_list:
    plt.plot(frequency_range,average_T_G[model_key],colors[model_key],linestyle=linestyles[model_key],label=model_title[model_key])
for model_key in model_list_param:
    plt.plot(frequency_range_param,average_T_G[model_key],colors[model_key],linestyle=linestyles[model_key],label=model_title[model_key])    
plt.fill_between(frequency_range,average_T_G["HaslamL"],average_T_G["HaslamU"],label=Haslam_label,color=colors["Haslam"],alpha=0.2,hatch='/')
#plt.scatter(frequencies_ITU, T_ITU, label=label_ITU, color=color_ITU, s=2.0)
plt.xlabel('Frequency / MHz', fontsize=25)
plt.ylabel(r'$\bar{T}$'+' / K', fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=25, ncol=2, columnspacing=0.5, handlelength=1.0, borderaxespad=0.3, labelspacing=0.4, borderpad=0.2)
plt.grid(False)
plt.xlim(30.,408.)
#plt.show()
plt.savefig(saveFolder+'AverageSkyTemperature/All_models.png', facecolor='white', bbox_inches='tight')
plt.savefig(saveFolder+'AverageSkyTemperature/pdf/All_models.pdf', facecolor='white', bbox_inches='tight')
plt.yscale('log')
plt.ylim(bottom=1e1, top=1e5)
#plt.show()
plt.savefig(saveFolder+'AverageSkyTemperature/All_models_log.png', facecolor='white', bbox_inches='tight')
plt.savefig(saveFolder+'AverageSkyTemperature/pdf/All_models_log.pdf', facecolor='white', bbox_inches='tight')
plt.xscale('log')
plt.savefig(saveFolder+'AverageSkyTemperature/All_models_loglog.png', facecolor='white', bbox_inches='tight')
plt.savefig(saveFolder+'AverageSkyTemperature/pdf/All_models_loglog.pdf', facecolor='white', bbox_inches='tight')
plt.show()


#Plot average sky temperature (normed)
plt.figure(figsize=(12,8))
for model_key in model_list:
    plt.plot(frequency_range,average_T_G[model_key]/average_T_G["HaslamU"],colors[model_key],linestyle=linestyles[model_key],label=model_title[model_key])
for model_key in model_list_param:
    plt.plot(frequency_range_param,average_T_G[model_key]/average_T_G["HaslamU"][0:71],colors[model_key],linestyle=linestyles[model_key],label=model_title[model_key])
plt.fill_between(frequency_range,average_T_G["HaslamL"]/average_T_G["HaslamU"],average_T_G["HaslamU"]/average_T_G["HaslamU"],label=Haslam_label,color=colors["Haslam"],alpha=0.2,hatch='/')
plt.xlabel('Frequency / MHz', fontsize=25)
#plt.ylabel('Average sky temperature \n (normalized to Haslam with '+r'$\beta$'+' = '+str(index_upper)+')', fontsize=28)
#plt.ylabel(r'$\frac{\bar{T}}{\bar{T}_{\mathrm{GSM (2008)}}}$', fontsize=28)
plt.ylabel(r'$\frac{\bar{T}}{\bar{T}_{\mathrm{Haslam, }\beta=\mathrm{%s}}}$'%(index_upper), fontsize=35)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=25, loc='upper right', borderaxespad=0.3, labelspacing=0.4, borderpad=0.2, ncol=2, columnspacing=0.5, handlelength=1.0)
plt.grid(False)
plt.xlim(30.,408.)
plt.axhline(1.0, color='grey')
#plt.show()
plt.savefig(saveFolder+'AverageSkyTemperature/All_models_normed.png', facecolor='white', bbox_inches='tight')
plt.savefig(saveFolder+'AverageSkyTemperature/pdf/All_models_normed.pdf', facecolor='white', bbox_inches='tight')
plt.yscale('log')
#plt.show()
plt.savefig(saveFolder+'AverageSkyTemperature/All_models_normed_log.png', facecolor='white', bbox_inches='tight')
plt.savefig(saveFolder+'AverageSkyTemperature/pdf/All_models_normed_log.pdf', facecolor='white', bbox_inches='tight')
plt.xscale('log')
plt.savefig(saveFolder+'AverageSkyTemperature/All_models_normed_loglog.png', facecolor='white', bbox_inches='tight')
plt.savefig(saveFolder+'AverageSkyTemperature/pdf/All_models_normed_loglog.pdf', facecolor='white', bbox_inches='tight')
plt.show()

#Plot global level-of-agreement as function of frequency
plt.figure(figsize=(12,4))
plt.plot(frequency_range, 100.*LoA)
plt.xlabel('Frequency / MHz', fontsize=25)
plt.ylabel(r'$r_\mathrm{max}$ / \%', fontsize=25)
#plt.ylabel(r'$max|\frac{T_\mathrm{sky,\; average}-T_{\mathrm{sky,\; average;\; Haslam,\;}\beta=2.46}}{T_\mathrm{sky,\; average}+T_{\mathrm{sky,\; average;\; Haslam,\;}\beta=2.46}}|$', fontsize=28)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
#plt.legend(fontsize=20)
plt.grid(False)
plt.xlim(30.,408.)
plt.savefig(saveFolder+'AverageSkyTemperature/All_models_LoA.png', facecolor='white', bbox_inches='tight')
plt.savefig(saveFolder+'AverageSkyTemperature/pdf/All_models_LoA.pdf', facecolor='white', bbox_inches='tight')
plt.plot(frequency_range, 100.*LoA_withoutLFSM)
plt.savefig(saveFolder+'AverageSkyTemperature/All_models_LoA_withoutLFSM.png', facecolor='white', bbox_inches='tight')
plt.savefig(saveFolder+'AverageSkyTemperature/pdf/All_models_LoA_withoutLFSM.pdf', facecolor='white', bbox_inches='tight')
plt.show()

#Plot average local sky temperature
plt.figure()
for model_key in model_list:
    average_T_L_mean = [np.mean(x) for x in average_T_L[model_key]]
    plt.plot(frequency_range,average_T_G[model_key],label=model_title[model_key], color=colors[model_key])
    plt.plot(frequency_range,average_T_L_mean,label=model_title[model_key]+' - local', color=colors[model_key], linestyle='--')
plt.xlabel('Frequency / MHz', fontsize=20)
plt.ylabel('Average sky temperature / K', fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(fontsize=20)
plt.grid(True)
plt.ylim(bottom=0.)
plt.savefig(saveFolder+'AverageSkyTemperature/All_models_local.png', facecolor='white', bbox_inches='tight')
plt.show()

# Plot 17h / 4h local T ratio
'''
plt.figure()
for model_key in model:
    average_T_L_17_4 = [a[np.where(hours==17.)]/a[np.where(hours==4.)] for a in average_T_L[model_key]]
    plt.plot(frequency_range,average_T_L_17_4,label=model_title[model_key]+' - local', color=colors[model_key], linestyle='--')
plt.xlabel('Frequency / MHz', fontsize=20)
plt.ylabel('Average sky temperature / K (17h / 4h)', fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(fontsize=20)
plt.grid(True)
plt.ylim(bottom=0.)
plt.savefig(saveFolder+'AverageSkyTemperature/All_models_local_17_4.png', facecolor='white', bbox_inches='tight')
plt.show()
'''

#Peak to Valley
plt.figure()
for model_key in model_list:
    average_T_L_PtV = [max(a)/min(a) for a in average_T_L[model_key]]
    plt.plot(frequency_range,average_T_L_PtV,label=model_title[model_key]+' - local', color=colors[model_key], linestyle='--')
plt.xlabel('Frequency / MHz', fontsize=20)
plt.ylabel('Average sky temperature / K (max / min)', fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(fontsize=20)
plt.grid(True)
plt.ylim(bottom=0.)
plt.savefig(saveFolder+'AverageSkyTemperature/All_models_local_PtV.png', facecolor='white', bbox_inches='tight')
plt.show()

#Plot local sky temperature over LST
indices = [0,1,2,3,4,5]

for i in indices:
    plt.figure()
    for model_key in model_list:
        plt.plot(hours,average_T_L[model_key][i],label=model_title[model_key]+' - local', color=colors[model_key], linestyle='--')
    plt.title(str(frequency_range[i])+' MHz', fontsize=20)
    plt.xlabel('LST / h', fontsize=20)
    plt.ylabel('Average sky temperature / K', fontsize=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.legend(fontsize=20)
    plt.grid(True)
    plt.ylim(bottom=0.)
    plt.savefig(saveFolder+'AverageSkyTemperature/All_models_local_LST_'+str(frequency_range[i]).replace('.','_')+'.png', facecolor='white', bbox_inches='tight')
    plt.show()
    
    plt.figure()
    for model_key in model_list:
        plt.plot(hours,average_T_L[model_key][i]/average_T_L['LFmap'][i],label=model_title[model_key]+' - local', color=colors[model_key], linestyle='--')
    plt.title(str(frequency_range[i])+' MHz', fontsize=20)
    plt.xlabel('LST / h', fontsize=20)
    plt.ylabel('Average sky temperature / K (normalized to LFmap)', fontsize=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.legend(fontsize=20)
    plt.grid(True)
    plt.ylim(bottom=0.)
    plt.savefig(saveFolder+'AverageSkyTemperature/All_models_local_LST_normed_'+str(frequency_range[i]).replace('.','_')+'.png', facecolor='white', bbox_inches='tight')
    plt.show()


for (m_1,m_2) in it.combinations(model_list,2):
    #Plot ratio of average sky temperature for two models
    plt.figure()
    plt.plot(frequency_range,100.*average_T_G[m_1]/average_T_G[m_2])
    plt.xlabel('Frequency / MHz', fontsize=20)
    plt.ylabel(r'Avg. sky temperature $\frac{\mathrm{%s}}{\mathrm{%s}}$'%(model_title[m_1],model_title[m_2])+' [%]', fontsize=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.grid(True)
    plt.savefig(saveFolder+'AverageSkyTemperature/Ratio_model_'+model_title[m_1]+'_'+model_title[m_2]+'.png', facecolor='white', bbox_inches='tight')
    plt.show()

    average_T_G_mean = [np.mean([average_T_G[m_1][i],average_T_G[m_2][i]]) for i in range(len(average_T_G[m_1]))]    
    #Plot difference of average sky temperature for two models
    plt.figure()
    plt.plot(frequency_range,100.*(average_T_G[m_2]-average_T_G[m_1])/average_T_G_mean)
    plt.xlabel('Frequency / MHz', fontsize=20)
    plt.ylabel(r'Avg. sky temperature $\frac{2(\mathrm{%s-%s})}{\mathrm{%s+%s}}$'%(model_title[m_2],model_title[m_1],model_title[m_2],model_title[m_1])+' [%]', fontsize=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.grid(True)
    plt.savefig(saveFolder+'AverageSkyTemperature/Difference_model_'+model_title[m_1]+'_'+model_title[m_2]+'.png', facecolor='white', bbox_inches='tight')
    plt.show()

    #Calculate mean average local sky temperatures over LST
    average_T_L_mean_1 = np.array([np.mean(x) for x in average_T_L[m_1]])
    average_T_L_mean_2 = np.array([np.mean(x) for x in average_T_L[m_2]])
    #Plot ratio of average local sky temperature for two models
    plt.figure()
    plt.plot(frequency_range,100.*average_T_L_mean_1/average_T_L_mean_2)
    plt.xlabel('Frequency / MHz', fontsize=20)
    plt.ylabel(r'Avg. local sky temperature $\frac{\mathrm{%s}}{\mathrm{%s}}$'%(model_title[m_1],model_title[m_2])+' [%]', fontsize=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.grid(True)
    plt.savefig(saveFolder+'AverageSkyTemperature/Ratio_model_'+model_title[m_1]+'_'+model_title[m_2]+'_local.png', facecolor='white', bbox_inches='tight')
    plt.show()

    average_T_L_mean = [np.mean([average_T_L[m_1][i],average_T_L[m_2][i]]) for i in range(len(average_T_L[m_1]))]    
    #Plot difference of average local sky temperature for two models
    plt.figure()
    plt.plot(frequency_range,100.*(average_T_L_mean_2-average_T_L_mean_1)/average_T_L_mean)
    plt.xlabel('Frequency / MHz', fontsize=20)
    plt.ylabel(r'Avg. local sky temperature $\frac{2(\mathrm{%s-%s})}{\mathrm{%s+%s}}$'%(model_title[m_2],model_title[m_1],model_title[m_2],model_title[m_1])+' [%]', fontsize=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.grid(True)
    plt.savefig(saveFolder+'AverageSkyTemperature/Difference_model_'+model_title[m_1]+'_'+model_title[m_2]+'_local.png', facecolor='white', bbox_inches='tight')
    plt.show()    

Reference maps zero level error relative to the average sky temperature

In [ ]:
# generate sky maps
frequency_range = [10., 22., 35., 38.,  40., 45., 50., 60., 70., 74., 80., 85., 150., 178., 408.]
zero_level_error = [2.0e4, 5.0e3, 10., 10., 10., 125., 10., 10., 10., 10., 10., 120., 40., 15., 3.]
zero_level_offset = [0.0, 0.0, 0.0, 0.0, 0.0, 544.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.46]
zero_level_offset_2 = [0.0, 0.0, 0.0, 0.0, 0.0, 160.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.0, 0.0]
zero_level_offset_3 = [0.0, 0.0, 0.0, 0.0, 0.0, 78.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0]

for i in range(len(frequency_range)):
    freq = frequency_range[i]
    zle = zero_level_error[i]
    zlo = zero_level_offset[i]
    zlo_2 = zero_level_offset_2[i]
    zlo_3 = zero_level_offset_3[i]
    print(freq, zle)
    print(GskyMapDF.iloc[0]['-90.0'],'\n')
    avg_T = []
    for model_key in model_list:
        g = model[model_key]
        map = g.generate(freq)
        if key in ["GSM", "GMOSS"]:
            map = map + 2.72548
        dumpG = newvisufunc.projview(map,coord=g_model_coord[model_key], return_only_data=True)
        GskyMapDF=convert2SkyDF(dumpG,full=True)
        avg_T.append(integrateOverAngles_cos(GskyMapDF, t_min=-90, t_max=90)/integrateOverAngles_cos(GskyMapDF/GskyMapDF, t_min=-90, t_max=90))
        print(model_key, round(100. * zle /avg_T[-1], 1), '%; ', round(100. * zlo /avg_T[-1], 1), '%; ', round(100. * zlo_2 /avg_T[-1], 1), '%; ', round(100. * zlo_3 /avg_T[-1], 1), '%')

    #print('NGP ?: ', round(100. * zle / GskyMapDF.iloc[0]['90.0'], 1), '%')
    #print('SGP ?: ', round(100. * zle / GskyMapDF.iloc[0]['-90.0'], 1), '%')
    print('\nAvg  : ',round(100. * zle / np.mean(avg_T), 1), '%; ', round(100. * zlo / np.mean(avg_T), 1), '%; ', round(100. * zlo_2 / np.mean(avg_T), 1), '%; ', round(100. * zlo_3 / np.mean(avg_T), 1), '%\n--------------------')
    #print('Avg  : ', round(100. * np.mean([zle / a for a in avg_T]), 1), '%')
    print(" ")

# Model Comparison with latitude variation

In [ ]:
average_T_G_temp, average_T_L_temp = {},{}

In [ ]:
model_list = ['ULSA_d'] #LFmap, "LFSM", "GSM", "GSM16"] #, "HaslamL", "HaslamU"
# rotate them to Local coordinates at some particular LST time
altitude = np.linspace(-90.,90.,19) #19
# generate sky maps
frequency_range = np.linspace(30.0,408.0,64) #64
hours = np.arange(0,24,3) #3
average_T_G, average_T_L = {},{}

#for model_key in model:
for model_key in model_list:
    g = model[model_key]
    map_title = model_title[model_key]
    T_G_temp,T_L_temp = [],[]
    for count_f, freq in enumerate(frequency_range):
        T_L_temp.append([])
        map = g.generate(freq)
        # first generate the data dump
        dumpG = newvisufunc.projview(map,coord=e_model_coord[model_key], return_only_data=True)

        # then convert the data dump to DataFrame
        # set full=True to have full Y axis
        GskyMapDF=convert2SkyDF(dumpG,full=True)
        print(model_key, freq)
        T_G_temp.append(integrateOverAngles_cos(GskyMapDF, t_min=90, t_max=90)/integrateOverAngles_cos(GskyMapDF/GskyMapDF, t_min=90, t_max=90))
        print(T_G_temp[-1])
        for lat in altitude:
            print(lat)
            T_hour = []
            for hour in hours:
                rotAngles=[(180+(hour*15))%360,-(lat-90)]
                dump = newvisufunc.projview(map,rot=rotAngles, coord=c_model_coord[model_key], return_only_data=True,xsize=newPhi.size)
                LskyMapDF=convert2SkyDF(dump,full=True)
                T_hour.append(integrateOverAngles_cos(LskyMapDF, t_min=0., t_max=90.)/integrateOverAngles_cos(LskyMapDF/LskyMapDF, t_min=0., t_max=90.))
            T_L_temp[count_f].append(T_hour)
    average_T_G[model_key] = np.array(T_G_temp)
    average_T_L[model_key] = np.array(T_L_temp)
    average_T_G_temp[model_key] = np.array(T_G_temp)
    average_T_L_temp[model_key] = np.array(T_L_temp)

In [ ]:
with open("averageT_lat_data_temp.pickle", "wb") as fout:
    pickle.dump([frequency_range, hours, average_T_G_temp, average_T_L_temp, altitude], fout)

In [ ]:
with open("averageT_lat_data_temp.pickle", "rb") as fin:
    frequency_range, hours, average_T_G_temp, average_T_L_temp, altitude = pickle.load(fin)

In [ ]:
with open("averageT_lat_data.pickle", "wb") as fout:
    pickle.dump([frequency_range, hours, average_T_G, average_T_L, altitude], fout)

In [ ]:
with open("averageT_lat_data.pickle", "rb") as fin:
    frequency_range, hours, average_T_G, average_T_L, altitude = pickle.load(fin)

In [ ]:
average_T_G_temp['LFmap'] = average_T_G['LFmap']
average_T_G_temp['GSM'] = average_T_G['GSM']
average_T_G_temp['GSM16'] = average_T_G['GSM16']
average_T_G_temp['LFSM'] = average_T_G['LFSM']
average_T_G_temp['GMOSS'] = average_T_G['GMOSS']
average_T_G_temp['SSM'] = average_T_G['SSM']
average_T_G_temp['ULSA'] = average_T_G['ULSA']

average_T_L_temp['LFmap'] = average_T_L['LFmap']
average_T_L_temp['GSM'] = average_T_L['GSM']
average_T_L_temp['GSM16'] = average_T_L['GSM16']
average_T_L_temp['LFSM'] = average_T_L['LFSM']
average_T_L_temp['GMOSS'] = average_T_L['GMOSS']
average_T_L_temp['SSM'] = average_T_L['SSM']
average_T_L_temp['ULSA'] = average_T_L['ULSA']

In [ ]:
average_T_G['LFmap'] = average_T_G_temp['LFmap']
average_T_G['LFSM'] = average_T_G_temp['LFSM']
average_T_G['GSM'] = average_T_G_temp['GSM']
average_T_G['GSM16'] = average_T_G_temp['GSM16']
average_T_G['GMOSS'] = average_T_G_temp['GMOSS']
average_T_G['SSM'] = average_T_G_temp['SSM']
average_T_G['ULSA'] = average_T_G_temp['ULSA']

average_T_L['LFmap'] = average_T_L_temp['LFmap']
average_T_L['LFSM'] = average_T_L_temp['LFSM']
average_T_L['GSM'] = average_T_L_temp['GSM']
average_T_L['GSM16'] = average_T_L_temp['GSM16']
average_T_L['GMOSS'] = average_T_L_temp['GMOSS']
average_T_L['SSM'] = average_T_L_temp['SSM']
average_T_L['ULSA'] = average_T_L_temp['ULSA']

In [ ]:
average_T_L

In [ ]:
average_T_L_temp

In [ ]:
average_T_G['GSM'] = average_T_G['GSM'] + 2.72548
average_T_G['GMOSS'] = average_T_G['GMOSS'] + 2.72548

average_T_L['GSM'] = average_T_L['GSM'] + 2.72548
average_T_L['GMOSS'] = average_T_L['GMOSS'] + 2.72548

In [ ]:
from matplotlib.ticker import StrMethodFormatter
plt.style.use('default')
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams['legend.title_fontsize'] = 20

model_list = ["LFmap", "GSM", "GSM16", "LFSM", "GMOSS", "SSM", "ULSA_d"]
Haslam_label = "Haslam "+str(index_lower)+r" $< \beta <$ "+str(index_upper)
colors = {"LFmap": "tab:blue", "LFSM": "tab:orange", "GSM": "tab:green", "GSM16": "tab:red", "Haslam": "grey", "Cane": "k", "Cane2": "m", 'GMOSS': 'tab:olive', 'SSM': 'lime', 'ULSA_d': 'tab:cyan'}
linestyles = {"LFmap": "solid", "LFSM": "solid", "GSM": "solid", "GSM16": "solid", "Haslam": "solid", "Cane": "dotted", "Cane2": "dotted", 'GMOSS': 'solid', 'SSM': 'solid', 'ULSA_d': 'solid'}
cmap = matplotlib.cm.get_cmap('viridis')


#Plot average local sky temperature
for model_key in model_list:
    plt.figure(figsize=(14,5))
    average_T_L_mean = np.mean(average_T_L[model_key], axis=-1)
    for i in range(average_T_L_mean.shape[1]):
        plt.plot(frequency_range,average_T_L_mean[:,i]/average_T_L_mean[:,9],label=str(altitude[i])+'°', color=cmap(i/average_T_L_mean.shape[1]), linestyle='--')
        
    plt.xlabel('Frequency / MHz', fontsize=20)
    plt.ylabel('Average sky temperature / K', fontsize=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    leg = plt.legend(title=model_key+r'$\\$'+'Geographic latitude', ncol=4, labelspacing=0.2, columnspacing=0.5, handlelength=1.0, fontsize=20)
    leg._legend_box.align = "left"
    plt.grid(True)
    plt.savefig(saveFolder+'AverageSkyTemperature/All_models_local_lat_'+model_key+'.png', facecolor='white', bbox_inches='tight')
    #plt.yscale('log')
    plt.savefig(saveFolder+'AverageSkyTemperature/All_models_local_lat_'+model_key+'_log.png', facecolor='white', bbox_inches='tight')
    plt.xscale('log')
    plt.xticks([30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400], ['30', '','', '', '', '80','','100','','300', ''] ,fontsize=16)
    plt.savefig(saveFolder+'AverageSkyTemperature/All_models_local_lat_'+model_key+'_loglog.png', facecolor='white', bbox_inches='tight')
    plt.savefig(saveFolder+'AverageSkyTemperature/pdf/All_models_local_lat_'+model_key+'_loglog.pdf', facecolor='white', bbox_inches='tight')
    plt.show()

#Plot average local sky temperature
fig, ax = plt.subplots(1, 1, figsize=(8,6))
fig_n, ax_n = plt.subplots(1, 1, figsize=(8,6))
for i, model_key in enumerate(model_list):
    average_T_L_mean = np.mean(average_T_L[model_key], axis=-1)
    T_temp = np.array([simps(average_T_L_mean[:,i],frequency_range) for i in range(average_T_L_mean.shape[1])])
    average_T_L_mean_norm = np.mean(average_T_L['GMOSS'], axis=-1)
    T_temp_norm = np.array([simps(average_T_L_mean_norm[:,i],frequency_range) for i in range(average_T_L_mean_norm.shape[1])])
    ax.plot(altitude, T_temp, label=model_title[model_key], color=colors[model_key], linestyle=linestyles[model_key])
    ax_n.plot(altitude, T_temp/T_temp_norm, label=model_title[model_key], color=colors[model_key], linestyle=linestyles[model_key])
        
ax.set_xlabel('Latitude', fontsize=25)
ax_n.set_xlabel('Latitude', fontsize=25)
ax.set_ylabel(r'$\mathcal{T}$'+' / K MHz', fontsize=25, labelpad=10)
ax_n.set_ylabel(r'\fontsize{30}{3em}\selectfont{}{$\frac{\mathcal{T}}{\mathcal{T}_\mathrm{GMOSS}}$}'+r'\fontsize{20}{3em}\selectfont{}{ / K '+r'\fontsize{20}{3em}\selectfont{}{MHz}', fontsize=25, labelpad=20)
ax.tick_params(axis='both',which='both', labelsize=20)
ax_n.tick_params(axis='both',which='both', labelsize=20)
ax.set_ylim(ymax=5.7e5)
ax.set_xlim(xmin=-90., xmax=90.)
ax_n.set_ylim(ymax=1.26) #ymin=0.995,
ax_n.set_xlim(xmin=-90., xmax=90.)
ax.set_yticks([4e5,5e5])
ax.set_yticklabels([r'$4\times10^5$',r'$5\times10^5$'])
ax.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}°'))
ax_n.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}°'))
#ax_n.yaxis.set_major_formatter(StrMethodFormatter('{x:.1f}'))
ax_n.ticklabel_format(axis='y', useOffset=False, style='plain')
ax.legend(fontsize=20, ncol=2, columnspacing=0.5, handlelength=1.0, borderaxespad=0.3, labelspacing=0.4, borderpad=0.2)
ax_n.legend(fontsize=20, ncol=2, columnspacing=0.5, handlelength=1.0, borderaxespad=0.3, labelspacing=0.4, borderpad=0.2)
ax.grid(False)
ax_n.grid(False)

fig.savefig(saveFolder+'AverageSkyTemperature/All_models_local_lat_combined.png', facecolor='white', bbox_inches='tight')
fig.savefig(saveFolder+'AverageSkyTemperature/pdf/All_models_local_lat_combined.pdf', facecolor='white', bbox_inches='tight')
ax.set_yscale('log')
fig.savefig(saveFolder+'AverageSkyTemperature/All_models_local_lat_combined_log.png', facecolor='white', bbox_inches='tight')
fig.show()

fig_n.savefig(saveFolder+'AverageSkyTemperature/All_models_local_lat_combined_normed.png', facecolor='white', bbox_inches='tight')
fig_n.savefig(saveFolder+'AverageSkyTemperature/pdf/All_models_local_lat_combined_normed.pdf', facecolor='white', bbox_inches='tight')
ax_n.set_yscale('log')
fig_n.savefig(saveFolder+'AverageSkyTemperature/All_models_local_lat_combined_log_normed.png', facecolor='white', bbox_inches='tight')
fig_n.show()



# Compare models for single radio experiments (specific geographical latitudes & frequency bands)

In [ ]:
model_list = ["LFmap", "LFSM", "GSM", "GSM16", "GMOSS", "SSM", 'ULSA_d'] # "LFmap", "LFSM", "GSM", "GSM16", "HaslamL", "HaslamU"
experiments = {"RNO-G": 72.58333, "LOFAR_low": 52.90889, "LOFAR_high": 52.90889, "GRAND": 42.9333, "OVRO-LWA": 37.2339, "SKA-low": -26.697, "Auger": -35.206667, "IceCube": -90.0}  # altitudes of RNO-G, LOFAR, GRANDproto, SKA-low, Auger, IceCube; sorted by altitude
# frequency bands of RNO-G: 100-1000 MHz, LOFAR low: 30-80 MHz, LOFAR high: 110-190 MHz, GRAND: 50-200 MHz, SKA-low: 50-350 MHz, Auger: 30-80 MHz, IceCube: 100-190 MHz
frequencies = {"RNO-G": np.linspace(100.0,408.0,78), "LOFAR_low": np.linspace(30.0,80.0,11), "LOFAR_high": np.linspace(110.0,190.0,21), "GRAND": np.linspace(50.0,200.0,26), "OVRO-LWA": np.linspace(30.0,80.0,11), "SKA-low": np.linspace(50.0,350.0,51), "Auger": np.linspace(30.0,80.0,11), "IceCube": np.linspace(70.0,350.0,41)}
hours = np.arange(0,24,2)
average_T_L = {}

for exp in experiments:
    average_T_L[exp] = {}
    print(exp)
    for model_key in model_list:
        g = model[model_key]
        map_title = model_title[model_key]
        T_L_temp = []
        for count_f, freq in enumerate(frequencies[exp]):
            T_L_temp.append([])
            map = g.generate(freq)
            print(model_key, freq)
            T_hour = []
            for hour in hours:
                rotAngles=[(180+(hour*15))%360,-(experiments[exp]-90)]
                dump = newvisufunc.projview(map,rot=rotAngles, coord=c_model_coord[model_key], return_only_data=True,xsize=newPhi.size)
                LskyMapDF=convert2SkyDF(dump,full=False)
                T_hour.append(integrateOverAngles_cos(LskyMapDF, t_min=15)/integrateOverAngles_cos(LskyMapDF/LskyMapDF, t_min=15))
            T_L_temp[count_f].append(T_hour)
        average_T_L[exp][model_key] = np.array(T_L_temp)

In [ ]:
with open("averageT_exp_data.pickle", "wb") as fout:
    pickle.dump([frequencies, hours, average_T_L, experiments], fout)

In [ ]:
with open("averageT_exp_data.pickle", "rb") as fin:
    frequencies, hours, average_T_L, experiments = pickle.load(fin)

In [ ]:
with open("averageT_exp_data_temp.pickle", "wb") as fout:
    pickle.dump([frequencies, hours, average_T_L_temp, experiments], fout)

In [ ]:
with open("averageT_exp_data_temp.pickle", "rb") as fin:
    frequencies, hours, average_T_L_temp, experiments = pickle.load(fin)

In [ ]:
model_list_temp = ["LFmap", "LFSM", "GSM", "GSM16", "GMOSS", "SSM", "ULSA_d"]
average_T_L_temp = {}
for exp in experiments:
    average_T_L_temp[exp] = {}
    for model_key in model_list_temp:
        average_T_L_temp[exp][model_key] = average_T_L[exp][model_key]

In [ ]:
model_list_temp = ["LFmap", "LFSM", "GSM", "GSM16", "GMOSS", "SSM", "ULSA_d"]
for exp in experiments:
    for model_key in model_list_temp:
        average_T_L[exp][model_key] = average_T_L_temp[exp][model_key]

In [ ]:
average_T_L

In [ ]:
average_T_L_temp

In [ ]:
# Add back CMB component
model_list_cmb = ["GSM", "GMOSS"]
for exp in experiments:
    for model_key in model_list_cmb:
        average_T_L[exp][model_key] = average_T_L[exp][model_key] + 2.72548

In [ ]:
plt.style.use('default')
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams['legend.title_fontsize'] = 20

model_list = {}
models = ["LFmap", "GSM", "GSM16", "LFSM", "GMOSS", "SSM", "ULSA_d"] #, "HaslamL", "HaslamU"

for exp in experiments:
    model_list[exp] = models
    res_temp = {}
    for (m_1,m_2) in it.combinations(model_list[exp],2):
        a1 = np.squeeze(np.mean(average_T_L[exp][m_1], axis=-1))
        a2 = np.squeeze(np.mean(average_T_L[exp][m_2], axis=-1))
        res_temp[m_1+' '+m_2] = round(100.0 * 2*(simps(a1,frequencies[exp])-simps(a2,frequencies[exp]))/(simps(a1,frequencies[exp])+simps(a2,frequencies[exp])),1)
    print("")
    print(exp)
    for el in res_temp:
        print(el, res_temp[el])
    print("Maximum: ", max(res_temp, key=lambda y: abs(res_temp[y])), res_temp[max(res_temp, key=lambda y: abs(res_temp[y]))], "%")

# Influence of Sun investigation by adding a hot circle to the map

In [ ]:
def F_to_T_sun(F, nu):
    c = 3.0e8
    k = 1.38e-23
    l = c/(nu*1e6)
    T = F*l**2 / (2* k * np.pi * (0.25*np.pi/180)**2)
    return T

In [ ]:
Sun_data = np.genfromtxt('Sun_Noise_2.txt', delimiter=';')
frequencies_sun, T_sun = Sun_data[Sun_data[:, 0].argsort()].T
#frequencies_sun *= 1e-6 only for Sun_Noise.txt

In [ ]:
from scipy.interpolate import interp1d
f = interp1d(frequencies_sun, T_sun, kind='linear')
x_new = np.linspace(15.51, 1.9e4, 100000)

plt.figure()
plt.plot(frequencies_sun, T_sun, 'k.', x_new, f(x_new), 'r')
plt.xscale('log')
plt.show()

In [ ]:
# rotate them to Local coordinates at some particular LST time
altitude = -35.206667 # Pierre Auger Observatory
#altitude = 52.90889 # LOFAR core
# generate sky maps
models = ["ULSA_d"]  #"LFmap", "LFSM", "GSM", "GSM16", 
frequency_range = np.linspace(30.0,408.0, 127) #127
average_T_E = {}
average_T_E_sun = {}

for model_key in models:
    e = model[model_key]
    map_title = model_title[model_key]
    T_E_temp, T_E_temp_sun = [],[]
    for freq in frequency_range:
        map = e.generate(freq)
        # first generate the data dump
        dumpE = newvisufunc.projview(map,coord=e_model_coord[model_key], return_only_data=True)

        # then convert the data dump to DataFrame
        # set full=True to have full Y axis
        EskyMapDF = convert2SkyDF(dumpE,full=True)
        EskyMapDF_sun = addSun(EskyMapDF, Temp=f(freq), radius=.25) #, cov=True
        print(model_key, freq)
        T_E_temp.append(integrateOverAngles_cos(EskyMapDF, t_min=-90, t_max=90)/integrateOverAngles_cos(EskyMapDF/EskyMapDF, t_min=-90, t_max=90))
        T_E_temp_sun.append(integrateOverAngles_cos(EskyMapDF_sun, t_min=-90, t_max=90)/integrateOverAngles_cos(EskyMapDF/EskyMapDF, t_min=-90, t_max=90))
        print(T_E_temp_sun[-1] - T_E_temp[-1])
    average_T_E[model_key] = np.array(T_E_temp)
    average_T_E_sun[model_key] = np.array(T_E_temp_sun)

In [ ]:
with open("averageT_sun_data.pickle", "wb") as fout:
    pickle.dump([frequency_range, average_T_E, average_T_E_sun], fout)

In [ ]:
with open("averageT_sun_data.pickle", "rb") as fin:
    frequency_range, average_T_E, average_T_E_sun = pickle.load(fin)

In [ ]:
average_T_E_temp = {}
average_T_E_sun_temp = {}

In [ ]:
average_T_E_temp['LFmap'] = average_T_E['LFmap']
average_T_E_temp['LFSM'] = average_T_E['LFSM']
average_T_E_temp['GSM'] = average_T_E['GSM']
average_T_E_temp['GSM16'] = average_T_E['GSM16']
average_T_E_temp['GMOSS'] = average_T_E['GMOSS']
average_T_E_temp['SSM'] = average_T_E['SSM']
average_T_E_temp['ULSA'] = average_T_E['ULSA']

average_T_E_sun_temp['LFmap'] = average_T_E_sun['LFmap']
average_T_E_sun_temp['LFSM'] = average_T_E_sun['LFSM']
average_T_E_sun_temp['GSM'] = average_T_E_sun['GSM']
average_T_E_sun_temp['GSM16'] = average_T_E_sun['GSM16']
average_T_E_sun_temp['GMOSS'] = average_T_E_sun['GMOSS']
average_T_E_sun_temp['SSM'] = average_T_E_sun['SSM']
average_T_E_sun_temp['ULSA'] = average_T_E_sun['ULSA']

In [ ]:
average_T_E['LFmap'] = average_T_E_temp['LFmap']
average_T_E['LFSM'] = average_T_E_temp['LFSM']
average_T_E['GSM'] = average_T_E_temp['GSM']
average_T_E['GSM16'] = average_T_E_temp['GSM16']
average_T_E['GMOSS'] = average_T_E_temp['GMOSS']
average_T_E['SSM'] = average_T_E_temp['SSM']
average_T_E['ULSA'] = average_T_E_temp['ULSA']

average_T_E_sun['LFmap'] = average_T_E_sun_temp['LFmap']
average_T_E_sun['LFSM'] = average_T_E_sun_temp['LFSM']
average_T_E_sun['GSM'] = average_T_E_sun_temp['GSM']
average_T_E_sun['GSM16'] = average_T_E_sun_temp['GSM16']
average_T_E_sun['GMOSS'] = average_T_E_sun_temp['GMOSS']
average_T_E_sun['SSM'] = average_T_E_sun_temp['SSM']
average_T_E_sun['ULSA'] = average_T_E_sun_temp['ULSA']

In [ ]:
average_T_E_temp

In [ ]:
average_T_E

In [ ]:
average_T_E['GSM'] = average_T_E['GSM'] + 2.72548
average_T_E['GMOSS'] = average_T_E['GMOSS'] + 2.72548

average_T_E_sun['GSM'] = average_T_E_sun['GSM'] + 2.72548
average_T_E_sun['GMOSS'] = average_T_E_sun['GMOSS'] + 2.72548

In [ ]:
plt.style.use('default')
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams['legend.title_fontsize'] = 20

model_list = ["LFmap", "GSM", "GSM16", "LFSM", "GMOSS", "SSM", "ULSA_d"]
Haslam_label = "Haslam "+str(index_lower)+r" $< \beta <$ "+str(index_upper)
colors = {"LFmap": "tab:blue", "LFSM": "tab:orange", "GSM": "tab:green", "GSM16": "tab:red", "Haslam": "grey", "Cane": "k", "Cane2": "m", 'GMOSS': 'tab:olive', 'SSM': 'lime', 'ULSA_d': 'tab:cyan'}
linestyles = {"LFmap": "solid", "LFSM": "solid", "GSM": "solid", "GSM16": "solid", "Haslam": "solid", "Cane": "dotted", "Cane2": "dotted", 'GMOSS': 'solid', 'SSM': 'solid', 'ULSA_d': 'solid'}

#Plot average local sky temperature
fig, ax = plt.subplots(1, 1, figsize=(10,7))
for model_key in model_list:
    reldif = 100. * (average_T_E_sun[model_key]-average_T_E[model_key])/average_T_E[model_key]
    ax.plot(frequency_range, reldif, label=model_title[model_key], color=colors[model_key], linestyle=linestyles[model_key])
        
ax.set_xlabel('Frequency / MHz', fontsize=25)
ax.set_ylabel(r'$\frac{\bar{T}^\mathrm{sun}-\bar{T}}{\bar{T}}$'+' /\%', fontsize=35)
ax.tick_params(axis='both',which='both', labelsize=20)
ax.legend(fontsize=25, columnspacing=0.5, handlelength=1.0, borderaxespad=0.3, labelspacing=0.4, borderpad=0.2)
ax.grid(False)

fig.savefig(saveFolder+'AverageSkyTemperature/All_models_sun.png', facecolor='white', bbox_inches='tight')
fig.savefig(saveFolder+'AverageSkyTemperature/pdf/All_models_sun.pdf', facecolor='white', bbox_inches='tight')
fig.show()

# Influence of the Sun for the selected radio arrays

In [ ]:
model_list = ["LFmap", "LFSM", "GSM", "GSM16", "GMOSS", "SSM", 'ULSA_d'] # "LFmap", "LFSM", "GSM", "GSM16", "HaslamL", "HaslamU"
experiments = {"OVRO-LWA": 37.2339} # "RNO-G": 72.58333, "LOFAR_low": 52.90889, "LOFAR_high": 52.90889, "GRAND": 42.9333, "OVRO-LWA": 37.2339, "SKA-low": -26.697, "Auger": -35.206667, "IceCube": -90.0}  # altitudes of RNO-G, LOFAR, GRANDproto, SKA-low, Auger, IceCube; sorted by altitude
# frequency bands of RNO-G: 100-1000 MHz, LOFAR low: 30-80 MHz, LOFAR high: 110-190 MHz, GRAND: 50-200 MHz, SKA-low: 50-350 MHz, Auger: 30-80 MHz, IceCube: 100-190 MHz
frequencies = {"RNO-G": np.linspace(100.0,408.0,78), "LOFAR_low": np.linspace(30.0,80.0,11), "LOFAR_high": np.linspace(110.0,190.0,21), "GRAND": np.linspace(50.0,200.0,26), "OVRO-LWA": np.linspace(30.0,80.0,11), "SKA-low": np.linspace(50.0,350.0,51), "Auger": np.linspace(30.0,80.0,11), "IceCube": np.linspace(70.0,350.0,41)}
#frequencies = {"RNO-G": np.linspace(100.0,408.0,78), "LOFAR_low": np.linspace(30.0,80.0,11), "LOFAR_high": np.linspace(110.0,190.0,19), "GRAND": np.linspace(50.0,200.0,26), "SKA-low": np.linspace(50.0,350.0,51), "Auger": np.linspace(30.0,80.0,11), "IceCube": np.linspace(70.0,350.0,41)}
#frequencies = {"LOFAR_high": np.linspace(110.0,190.0,21)}
#frequencies = {"RNO-G": np.linspace(100.0,408.0,2), "LOFAR_low": np.linspace(30.0,80.0,2), "LOFAR_high": np.linspace(110.0,200.0,2), "GRAND": np.linspace(50.0,200.0,2), "SKA-low": np.linspace(50.0,350.0,2), "Auger": np.linspace(30.0,80.0,2), "IceCube": np.linspace(70.0,350.0,2)}
hours = np.arange(0,24,2) #2


# rotate them to Local coordinates at some particular LST time
# generate sky maps
average_T_L = {}
average_T_L_sun = {}

for exp in experiments:
    average_T_L[exp] = {}
    average_T_L_sun[exp] = {}
    print(exp)
    for model_key in model_list:
        g = model[model_key]
        map_title = model_title[model_key]
        T_L_temp, T_L_temp_sun = [],[]
        for count_f, freq in enumerate(frequencies[exp]):
            T_L_temp.append([])
            T_L_temp_sun.append([])
            map = g.generate(freq)
            # first generate the data dump
            T_hour, T_sun_hour = [], []
            for hour in hours:
                rotAngles=[(180+(hour*15))%360,-(experiments[exp]-90)]
                dumpL = newvisufunc.projview(map,rot=rotAngles, coord=c_model_coord[model_key], return_only_data=True, xsize=newPhi.size)

                # then convert the data dump to DataFrame
                # set full=True to have full Y axis
                LskyMapDF = convert2SkyDF(dumpL,full=True)
                LskyMapDF_sun = addSun_Local(LskyMapDF, Temp=f(freq), decl=.75, radius=.25)
                print(model_key, freq)
                T_hour.append(integrateOverAngles_cos(LskyMapDF, t_min=15, t_max=90)/integrateOverAngles_cos(LskyMapDF/LskyMapDF, t_min=15, t_max=90))
                T_sun_hour.append(integrateOverAngles_cos(LskyMapDF_sun, t_min=15, t_max=90)/integrateOverAngles_cos(LskyMapDF/LskyMapDF, t_min=15, t_max=90))
            T_L_temp[count_f].append(T_hour)
            T_L_temp_sun[count_f].append(T_sun_hour)
            #print(T_L_temp_sun[-1] - T_L_temp[-1])
        average_T_L[exp][model_key] = np.array(T_L_temp)
        average_T_L_sun[exp][model_key] = np.array(T_L_temp_sun)

In [ ]:
with open("averageT_L_sun_data.pickle", "wb") as fout:
    pickle.dump([model_list, experiments, frequencies, hours, average_T_L, average_T_L_sun], fout)

In [ ]:
with open("averageT_L_sun_data.pickle", "rb") as fin:
    model_list, experiments, frequencies, hours, average_T_L, average_T_L_sun = pickle.load(fin)

In [ ]:
with open("averageT_L_sun_data_2.pickle", "wb") as fout:
    pickle.dump([model_list, experiments, frequencies, hours, average_T_L, average_T_L_sun], fout)

In [ ]:
with open("averageT_L_sun_data_2.pickle", "rb") as fin:
    model_list, experiments, frequencies, hours, average_T_L, average_T_L_sun = pickle.load(fin)

In [ ]:
average_T_L_temp = average_T_L
average_T_L_sun_temp = average_T_L_sun

In [ ]:
model_list_temp = ["LFmap", "LFSM", "GSM", "GSM16", "GMOSS", "SSM", "ULSA"] #, "SSM", "ULSA"
average_T_L_temp = {}
average_T_L_sun_temp = {}
for exp in experiments:
    average_T_L_temp[exp] = {}
    average_T_L_sun_temp[exp] = {}
    for model_key in model_list_temp:
        average_T_L_temp[exp][model_key] = average_T_L[exp][model_key]
        average_T_L_sun_temp[exp][model_key] = average_T_L_sun[exp][model_key]

In [ ]:
model_list_temp = ["LFmap", "LFSM", "GSM", "GSM16", "GMOSS", "SSM", "ULSA_d"]
experiments = {"RNO-G": 72.58333, "LOFAR_low": 52.90889, "LOFAR_high": 52.90889, "GRAND": 42.9333, "SKA-low": -26.697, "Auger": -35.206667, "IceCube": -90.0}
for exp in experiments:
    average_T_L[exp] = {}
    average_T_L_sun[exp] = {}
    for model_key in model_list_temp:
        average_T_L[exp][model_key] = average_T_L_temp[exp][model_key]
        average_T_L_sun[exp][model_key] = average_T_L_sun_temp[exp][model_key]

In [ ]:
model_list = ["LFmap", "LFSM", "GSM", "GSM16", 'GMOSS', 'SSM', 'ULSA_d'] # "HaslamL", "HaslamU"

In [ ]:
average_T_L_sun

In [ ]:
average_T_L_temp

In [ ]:
model_list_cmb = ["GSM", "GMOSS"]
experiments = {"RNO-G": 72.58333, "LOFAR_low": 52.90889, "LOFAR_high": 52.90889, "GRAND": 42.9333, "OVRO-LWA": 37.2339, "SKA-low": -26.697, "Auger": -35.206667, "IceCube": -90.0}
for exp in experiments:
    for model_key in model_list_cmb:
        average_T_L[exp][model_key] = average_T_L[exp][model_key] + 2.72548
        average_T_L_sun[exp][model_key] = average_T_L_sun[exp][model_key] + 2.72548

In [ ]:
from matplotlib.ticker import StrMethodFormatter
plt.style.use('default')
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams['legend.title_fontsize'] = 20

experiments = {"GRAND": 42.9333, "Auger": -35.206667, "LOFAR_low": 52.90889, "LOFAR_high": 52.90889, "RNO-G": 72.58333, "SKA-low": -26.697, "IceCube": -90.0, "OVRO-LWA": 37.2339}
ax = {"RNO-G": 1, "LOFAR_low": 0, "LOFAR_high": 0, "GRAND": 0, "SKA-low": 1, "Auger": 0, "IceCube": 1, "OVRO-LWA": 0}
colors = {"RNO-G": 'tab:orange', "LOFAR_low": 'tab:red', "LOFAR_high": 'tab:red', "GRAND": 'tab:cyan', "SKA-low": 'tab:red', "Auger": 'tab:green', "IceCube": 'tab:brown', "OVRO-LWA": 'yellow'}
linestyles = {"RNO-G": '-', "LOFAR_low": '-.', "LOFAR_high": ':', "GRAND": '-', "SKA-low": ':', "Auger": '--', "IceCube": '-.', "OVRO-LWA": 'solid'}
labels = {"RNO-G": 'RNO-G', "LOFAR_low": 'LOFAR low band', "LOFAR_high": 'LOFAR high band', "GRAND": 'GRAND', "SKA-low": 'SKA-low', "Auger": 'Auger', "IceCube": 'IceCube', "OVRO-LWA": "OVRO-LWA"}
zorders = {"RNO-G": 2, "LOFAR_low": 3, "LOFAR_high": 2, "GRAND": 0, "SKA-low": 0, "Auger": 1, "IceCube": 1, "OVRO-LWA": 4}
fig, axs = plt.subplots(2, 1, figsize=(7,10), constrained_layout = True) #sharex=True ; for subplots(1,2) use figsize (14,5)

for exp in experiments:
    temp_T_L, temp_T_L_sun = [],[]
    for model_key in model_list:
        temp_T_L.append(np.squeeze(np.mean(average_T_L[exp][model_key], axis=-1)))
        temp_T_L_sun.append(np.squeeze(np.mean(average_T_L_sun[exp][model_key], axis=-1)))
    temp_T_L_max = np.asarray(np.max(temp_T_L, axis=0))
    temp_T_L_min = np.asarray(np.min(temp_T_L, axis=0))
    temp_T_L = np.asarray(np.mean(temp_T_L, axis=0))
    temp_T_L_sun_max = np.asarray(np.max(temp_T_L_sun, axis=0))
    temp_T_L_sun_min = np.asarray(np.min(temp_T_L_sun, axis=0))
    temp_T_L_sun = np.asarray(np.mean(temp_T_L_sun, axis=0))
    reldif = 100. * (temp_T_L_sun-temp_T_L)/temp_T_L
    reldif_max = 100. * (temp_T_L_sun_max-temp_T_L_max)/temp_T_L_max
    reldif_min = 100. * (temp_T_L_sun_min-temp_T_L_min)/temp_T_L_min
    axs[ax[exp]].plot(frequencies[exp], reldif, color=colors[exp], linestyle=linestyles[exp], label=labels[exp], zorder=zorders[exp])
    axs[ax[exp]].fill_between(frequencies[exp], reldif_max, reldif_min, color=colors[exp], zorder=zorders[exp], alpha=0.1)

#axs[0].set_xlabel('Frequency / MHz', fontsize=20)
axs[1].set_xlabel('Frequency / MHz', fontsize=20)
axs[0].set_ylabel(r'$\frac{\bar{T}^\mathrm{sun}_\mathrm{local}-\bar{T}_\mathrm{local}}{\bar{T}_\mathrm{local}}$', fontsize=28, labelpad=14) #+r'\fontsize{19}{3em}\selectfont{}{/\%}'
axs[1].set_ylabel(r'$\frac{\bar{T}^\mathrm{sun}_\mathrm{local}-\bar{T}_\mathrm{local}}{\bar{T}_\mathrm{local}}$', fontsize=28, labelpad=6) #+r'\fontsize{19}{3em}\selectfont{}{/\%}'
#axs[1].set_ylabel(r'$\frac{T^\mathrm{sun}_\mathrm{sky, average}-T_\mathrm{sky, average}}{T_\mathrm{sky, average}}$'+' /\%', fontsize=24)
#axs[1].set_xlim(xmin=30, xmax=408)
axs[0].tick_params(axis='both',which='both', labelsize=16)
axs[1].tick_params(axis='both',which='both', labelsize=16)
axs[0].yaxis.set_major_formatter(StrMethodFormatter('{x}\%'))
axs[1].yaxis.set_major_formatter(StrMethodFormatter('{x}\%'))
#axs[0].set_xscale('log')
#axs[1].set_xscale('log')
axs[0].legend(fontsize=20, columnspacing=0.5, handlelength=2.0, borderaxespad=0.3, labelspacing=0.4, borderpad=0.2) # framealpha=0.5
axs[1].legend(fontsize=20, loc=2, columnspacing=0.5, handlelength=2.0, borderaxespad=0.3, labelspacing=0.4, borderpad=0.2)
#axs[0].grid(True)
#axs[1].grid(True)

fig.savefig(saveFolder+'AverageSkyTemperature/All_models_sun_Local.png', facecolor='white', bbox_inches='tight')
fig.savefig(saveFolder+'AverageSkyTemperature/pdf/All_models_Local_sun.pdf', facecolor='white', bbox_inches='tight')
fig.show()

# Convolution of the Antenna pattern with the sky temperature

Import some modules.

In [ ]:
from scipy import constants
from scipy.integrate import simps

The simple function <i>foldSkyWithAntenna</i> will multiply the squared antenna pattern with the sky temperature. The antenna pattern will be transfomed to healpy convention and further it will be interpolated at the angles of the sky temperature map.

In [ ]:
#skyMapDF = skyMapDF_LFSS
def foldSkyWithAntenna(antenna, frequency=None,orientation=None,skyMapDF=None):
    quantity = 'absHeight'
    p = skyMapDF.index.values.astype(float)
    t = skyMapDF.columns.values.astype(float)
    Amp = antenna.get(quantity=quantity, frequency=frequency, newTheta=t, newPhi=p, kind='cubic', changeConvention=True,
                      orientation=orientation)
    foldedAntennaSkyMap = Amp**2*skyMapDF
    return foldedAntennaSkyMap

First we need to generate sky temperature maps at some frequency F.

In [ ]:
# first generate maps at some frequeny
map_1 = g_1.generate(frequency)
map_2 = g_2.generate(frequency)

Set the altitude and local sidereal time at the local position where the antenna is (PAO).

In [ ]:
# rotate them to Local coordinates at some particular LST time
LSTtime = 18
altitude = -35.206667

Rotated and dump the data.

In [ ]:
rotAngles=[(180+(LSTtime*15))%360,-(altitude-90)]
dump_1 = newvisufunc.projview(map_1,rot=rotAngles, coord=c_coord_1,return_only_data=True,xsize=1439)
dump_2 = newvisufunc.projview(map_2,rot=rotAngles, coord=c_coord_2, return_only_data=True,xsize=1439)

Now, convert the data dump to the DataFrame and then use the folding function. We can do this in one step by nesting the functions.

In [ ]:
foldedAntennaSkyMap_1 = foldSkyWithAntenna(antenna, frequency=frequency, orientation='EW', skyMapDF=convert2SkyDF(dump_1,full=False))
foldedAntennaSkyMap_2 = foldSkyWithAntenna(antenna, frequency=frequency, orientation='EW', skyMapDF=convert2SkyDF(dump_2,full=False))

In [ ]:
orientation='EW'

## Plot of the "antenna-sky temperature" convolution at 18:00 LST
Plot the results of the convolution and compare them by dividing one by the other one.

In [ ]:
P, T = np.meshgrid(foldedAntennaSkyMap_1.index.values.astype(float), foldedAntennaSkyMap_1.columns.values.astype(float))

save=False
save1= saveFolder+"foldedAntennaSkyMap_"+map_1_title+"_"+orientation+"_LST_"+str(LSTtime)
save2= saveFolder+"foldedAntennaSkyMap_"+map_2_title+"_"+orientation+"_LST_"+str(LSTtime)
save3= saveFolder+"Ratio_foldedAntennaSkyMap_"+map_1_title+"-foldedAntennaSkyMap_"+map_2_title+"_"+ orientation+"_LST_"+str(LSTtime)
                                                                                                    
# log here!
myPlots.plot3dnewV3(P, T, np.log10(foldedAntennaSkyMap_1.values.T),
                    slices=True, ymax=90.5, xMajorLocator=30, yMajorLocator=15, figureWidth=12, figureHeight=8,
                    xlabel='azimuth$\degree$', ylabel='altitude$\degree$', cbarLabel='Temperature log[K]', extend='both',
                    mainTitle=map_1_title+': '+orientation+" orientation at frequency "+str(frequency)+\
                    ' MHz at '+str(LSTtime)+' LST\n Squared Antenna pattern convoluted with sky temperature',Cmin=3.25, Cmax=5.3,
                    save=save1,close=close)

myPlots.plot3dnewV3(P, T, np.log10(foldedAntennaSkyMap_2.values.T),
                    slices=True, ymax=90.5, xMajorLocator=30, yMajorLocator=15, figureWidth=12, figureHeight=8,
                    xlabel='azimuth$\degree$', ylabel='altitude$\degree$', cbarLabel='Temperature log[K]', extend='both',
                    mainTitle=map_2_title+': '+orientation+" orientation at frequency "+str(frequency)+\
                    ' MHz at '+str(LSTtime)+' LST\n Squared Antenna pattern convoluted with sky temperature'
                    ,Cmin=3.25, Cmax=5.3,
                    save=save2,close=close)

ratio= foldedAntennaSkyMap_1.values.T/foldedAntennaSkyMap_2.values.T
myPlots.plot3dnewV3(P, T, ratio,
                    slices=True, ymax=90.5, xMajorLocator=30, yMajorLocator=15, figureWidth=12, figureHeight=8,
                    xlabel='azimuth$\degree$', ylabel='altitude$\degree$', cbarLabel=r'$\frac{\mathrm{%s}}{\mathrm{%s}}$'%(map_1_title, map_2_title),
                    mainTitle='Ratio '+map_1_title+' and '+map_2_title+': '+orientation+" orientation at frequency "+str(frequency)+\
                    ' MHz at '+str(LSTtime)+' LST\n Squared Antenna pattern convoluted with sky temperature',
                    Cmin=0,Cmax=2,extend='max', save=save3,close=close)

## "Antenna-sky temperature" convolution at 02:00 LST
Same example as before but done for 2:00 LST.

In [ ]:
LSTtime=2

In [ ]:
rotAngles=[(180+(LSTtime*15))%360,-(altitude-90)]
dump_1 = newvisufunc.projview(map_1,rot=rotAngles, coord=c_coord_1,return_only_data=True, xsize=1439)
dump_2 = newvisufunc.projview(map_2,rot=rotAngles, coord=c_coord_2, return_only_data=True, xsize=1439)

In [ ]:
# first convert the data dump to DF and then use the folding function
foldedAntennaSkyMap_1 = foldSkyWithAntenna(antenna, frequency=frequency, orientation='EW',skyMapDF=convert2SkyDF(dump_1,full=False))
foldedAntennaSkyMap_2 = foldSkyWithAntenna(antenna, frequency=frequency, orientation='EW',skyMapDF=convert2SkyDF(dump_2,full=False))
orientation='EW'

In [ ]:
P, T = np.meshgrid(foldedAntennaSkyMap_1.index.values.astype(float),foldedAntennaSkyMap_1.columns.values.astype(float))

save=False
save1= saveFolder+"foldedAntennaSkyMap_"+map_1_title+"_"+orientation+"_LST_"+str(LSTtime)
save2= saveFolder+"foldedAntennaSkyMap_"+map_2_title+"_"+orientation+"_LST_"+str(LSTtime)
save3= saveFolder+"Ratio_foldedAntennaSkyMap_"+map_1_title+"-foldedAntennaSkyMap_"+map_2_title+"_"+ orientation+"_LST_"+str(LSTtime)
                                                                                                    
# log here!
myPlots.plot3dnewV3(P, T, np.log10(foldedAntennaSkyMap_1.values.T),
                    slices=True, ymax=90.5, xMajorLocator=30,yMajorLocator=15,figureWidth=12,figureHeight=8,
                    xlabel='azimuth$\degree$', ylabel='altitude$\degree$', cbarLabel='Temperature log[K]', extend='both',
                    mainTitle=map_1_title+': '+orientation+" orientation at frequency "+str(frequency)+\
                    ' MHz at '+str(LSTtime)+' LST\n Squared Antenna pattern convoluted with sky temperature', Cmin=3.25, Cmax=5.3,
                    save=save1,close=close)

myPlots.plot3dnewV3(P, T, np.log10(foldedAntennaSkyMap_2.values.T),
                    slices=True, ymax=90.5, xMajorLocator=30, yMajorLocator=15, figureWidth=12, figureHeight=8,
                    xlabel='azimuth$\degree$', ylabel='altitude$\degree$', cbarLabel='Temperature log[K]', extend='both',
                    mainTitle= map_2_title+': '+orientation+" orientation at frequency "+str(frequency)+\
                    ' MHz at '+str(LSTtime)+' LST\n Squared Antenna pattern convoluted with sky temperature',
                    Cmin=3.25, Cmax=5.3,
                    save=save2,close=close)

ratio=foldedAntennaSkyMap_1.values.T/foldedAntennaSkyMap_2.values.T
myPlots.plot3dnewV3(P, T, ratio,
                    slices=True, ymax=90.5, xMajorLocator=30, yMajorLocator=15, figureWidth=12, figureHeight=8,
                    xlabel='azimuth$\degree$', ylabel='altitude$\degree$', cbarLabel=r'$\frac{\mathrm{%s}}{\mathrm{%s}}$'%(map_1_title, map_2_title),
                    mainTitle='Ratio '+map_1_title+' and '+map_2_title+': '+orientation+" orientation at frequency "+str(frequency)+\
                    ' MHz at '+str(LSTtime)+' LST\n Squared Antenna pattern convoluted with sky temperature',
                    Cmin=0, Cmax=2, extend='max', save=save3, close=close)

## Integral over the angles
What is of interest for the calibration is the integral over the angles: <br>
$\int_{\Omega} T_{sky}(t,f,\theta,\phi) |H(f,\theta,\phi)|^{2} d\Omega$

Define a function for the integration <i>integrateOverAngles</i>

In [ ]:
# integrate the folded sky map over the angles
def integrateOverAngles(foldedAntennaSkyMap):
    p = np.deg2rad(foldedAntennaSkyMap.index.values.astype(float))
    t = np.deg2rad(foldedAntennaSkyMap.columns.values.astype(float))
    T, P = np.meshgrid(t, p)
    angePrefactor=np.sin(T[:,::-1])
    integratedOverAngles=simps(simps(foldedAntennaSkyMap*angePrefactor,t),p)
    return integratedOverAngles

As was showed before, first generate the rotated maps, dump the result, convert to the DataFrame and finally fold the antenna with the sky temperature maps.

In [ ]:
LSTtime=18
rotAngles = [(180+(LSTtime*15))%360,-(altitude-90)]
dump_1 = newvisufunc.projview(map_1, rot=rotAngles, coord=c_coord_1, return_only_data=True, xsize=1439)
dump_2 = newvisufunc.projview(map_2, rot=rotAngles, coord=c_coord_2, return_only_data=True, xsize=1439)
foldedAntennaSkyMap_1 = foldSkyWithAntenna(antenna, frequency=frequency, orientation='EW', skyMapDF=convert2SkyDF(dump_1,full=False))
foldedAntennaSkyMap_2 = foldSkyWithAntenna(antenna, frequency=frequency, orientation='EW', skyMapDF=convert2SkyDF(dump_2,full=False))

Once we have the convoluted maps we can integrate them.

In [ ]:
angle_integrated_1 = integrateOverAngles(foldedAntennaSkyMap_1)
angle_integrated_2 = integrateOverAngles(foldedAntennaSkyMap_2)

Print the values and their ratio.

In [ ]:
print(angle_integrated_1)
print(angle_integrated_2)
print('Ratio: ',angle_integrated_1/angle_integrated_2)

Try this for different LST, for example 2:00

In [ ]:
LSTtime=2
rotAngles=[(180+(LSTtime*15))%360,-(altitude-90)]
dump_1 = newvisufunc.projview(map_1,rot=rotAngles, coord=c_coord_1, return_only_data=True,xsize=1439)
dump_2 = newvisufunc.projview(map_2,rot=rotAngles, coord=c_coord_2, return_only_data=True,xsize=1439)
foldedAntennaSkyMap_1 = foldSkyWithAntenna(antenna, frequency=45, orientation='EW', skyMapDF=convert2SkyDF(dump_1,full=False))
foldedAntennaSkyMap_2 = foldSkyWithAntenna(antenna, frequency=45, orientation='EW', skyMapDF=convert2SkyDF(dump_2,full=False))

In [ ]:
angle_integrated_1 = integrateOverAngles(foldedAntennaSkyMap_1)
angle_integrated_2 = integrateOverAngles(foldedAntennaSkyMap_2)
print(angle_integrated_1)
print(angle_integrated_2)
print('Ratio: ',angle_integrated_1/angle_integrated_2)

# Power and Power spectral density calculation
Once we have the integrated antenna-sky temperature term it is easy to calculate the power spectral density. 

The power spectrum density is therefore: <br>

 \begin{equation*} 
\mathscr{P}_{sky}(t,f) = \frac{k_{b}}{c^{2}} f^{2} \int_{\Omega} T_{sky}(t,f,\theta,\phi) \frac{|H(f,\theta,\phi)|^{2}Z_{0}}{R_{r}} d\Omega           \end{equation*} <br>

where <br>
 \begin{equation*}
 <|H(f,\theta,\phi)|^{2}> = \frac{1}{2}(|J_{\theta}|^2+|J_{\phi}|^2|)
\end{equation*} <br>

The term <br>

 \begin{equation*}
\int_{\Omega} T_{sky}(t,f,\theta,\phi) |H(f,\theta,\phi)|^{2}  d \Omega
\end{equation*} <br>

is the integral of antenna-sky temperature that was calculated before.

To get the Power we need to just integrate the power spectrum density over the frequency. <br>

 \begin{equation*}
P_{sky}(t,f) = \int_{f} \mathscr{P}_{sky}(t,f) df
\end{equation*} <br>

Thus, the full equation for the Sky Power Radio Emmision is: <br>

 \begin{equation*}
 P_{sky}(t,f) = \frac{k_{b}}{c^{2}} \int_{f} f^{2} \int_{\Omega} T_{sky}(t,f,\theta,\phi) \frac{|H(f,\theta,\phi)|^{2}Z_{0}}{R_{r}} df d\Omega  
\end{equation*} <br>

where $Z_0$ is the vacuum impedance, $R_r$ is the antenna impedance, $k_b$ the Boltzmann constant and $c$ is the speed of light.

***

<p style='text-align: justify;'>
In the following cell are all necessery functions for the power calculation. Each of them was presented and described previously. The only new function is <i>simulateGalacticModulationCurve</i> that is calculating the power spectrum density for the chose frequency and LST. By default a DataFrame for frequencies from 30 to 80 MHz over the whole sidereal day is generated. The default frequency and LST step is 1.
</p>

In [ ]:
# this function helps to convert the data dump from newvisufunc.projview() to DF
def convert2SkyDF(data_dump,full=False):
    def DFtemplateCreator(xs,ys,yName=None):
        colList = [str(x) for x in xs]
        nan_init = np.empty((len(xs),len(ys),))
        nan_init[:]=np.nan
        templateDF = pd.DataFrame(nan_init.T, columns = colList)
        templateDF.insert(0,yName,ys)
        templateDF = templateDF.set_index(yName)
        return templateDF
    longitude,latitude,grid_map = data_dump
    latitudeDeg = np.rad2deg(latitude)
    longitudeDeg = np.rad2deg(longitude)
    skyMapDF = DFtemplateCreator(latitudeDeg,longitudeDeg,yName=None)
    skyMapDF.iloc[:,:] = grid_map.T
    if full is False:
        skyMapDF = skyMapDF.iloc[:,np.where(skyMapDF.columns.astype(float) >=0)[0].tolist()]
    skyMapDF=skyMapDF.sort_index()
    return skyMapDF

def DFtemplateCreator(xs,ys,yName=None):
    colList = [str(x) for x in xs]
    nan_init = np.empty((len(xs),len(ys),))
    nan_init[:]=np.nan
    templateDF = pd.DataFrame(nan_init.T, columns = colList)
    templateDF.insert(0,yName,ys)
    templateDF = templateDF.set_index(yName)
    return templateDF

def simulateGalacticModulationCurve(antennaClass=None, skyMapClass=None, startFrequency=30.0,endFrequency=80.0,frequencySpacing=2.0,
                                    startLST=0,endLST=24,LSTspacing=2, coord=['C'],
                                    orientation=None,returnOnlyIntegratedAngle=False, impedanceFile=None):
    Z0 = constants.physical_constants["characteristic impedance of vacuum"][0]
    frequencies = np.arange(startFrequency,endFrequency+frequencySpacing,frequencySpacing)
    LSTtimes = np.arange(startLST,endLST,LSTspacing)
    if impedanceFile is not None:
        impedanceData = np.loadtxt(impedanceFile)
        impedanceFunction = interp1d(impedanceData[0],impedanceData[1])
    else:
        print('No impedance file! Impedance set to 1')
    if antennaClass == None:
        print('Pass the antenna class please.')
        return None
    if skyMapClass == None:
        print('Pass the sky map class please.')
        return None
    if orientation == None:
        print('Orientation? EW or NS?')
        return None
    altitude = -35.206667
    inc = 0.1
    angle_integratedList=[]
    powerSpectrumDensityList=[]
    constantFactor = (2*constants.k/constants.c**2)*Z0
    for LSTtime in tqdm(LSTtimes):
        angle_integratedList_at_freq_F=[]
        powerSpectrumDensityList_at_freq_F=[]
        for frequency in frequencies:
            if impedanceFile is not None:
                freqDepPrefactor = ((frequency*1e+6)**2)/impedanceFunction(frequency)
            else:
                freqDepPrefactor = (frequency*1e+6)**2
            #print('I AM AT FREQUENCY: '+str(frequency)+' Mhz.')
            skyMap = skyMapClass.generate(frequency)
            rotAngles=[(180+(LSTtime*15))%360,-(altitude-90)]
            # xsize=1439 will give angle spacing of 0.25 degrees
            dump = newvisufunc.projview(skyMap,rot=rotAngles, coord=coord, return_only_data=True,xsize=1439)
            foldedAntennaSkyMap = foldSkyWithAntenna(antennaClass, frequency=frequency, orientation=orientation, skyMapDF=convert2SkyDF(dump, full=False))
            angle_integrated = integrateOverAngles(foldedAntennaSkyMap)
            angle_integratedList_at_freq_F.append(angle_integrated)
            # power spectrum density
            powerSpectrumDensity = constantFactor*freqDepPrefactor*angle_integrated
            powerSpectrumDensityList_at_freq_F.append(powerSpectrumDensity)
        angle_integratedList.append(np.asarray(angle_integratedList_at_freq_F))
        powerSpectrumDensityList.append(np.asarray(powerSpectrumDensityList_at_freq_F))
    if returnOnlyIntegratedAngle == True:
        angle_integratedDF = DFtemplateCreator(frequencies,LSTtimes,yName=None)
        angle_integratedDF.iloc[:,:] = np.asarray(angle_integratedList)
        return angle_integratedDF
    else:
        powerSpectrumDensityDF = DFtemplateCreator(frequencies,LSTtimes,yName=None)
        powerSpectrumDensityDF.iloc[:,:]= np.asarray(powerSpectrumDensityList)
        return powerSpectrumDensityDF

#skyMapDF = skyMapDF_LFSS
def foldSkyWithAntenna(antenna, frequency=None,orientation=None,skyMapDF=None):
    quantity = 'absHeight'
    p = skyMapDF.index.values.astype(float)
    t = skyMapDF.columns.values.astype(float)
    Amp = antenna.get(quantity=quantity, frequency=frequency, newTheta=t, newPhi=p, kind='linear', changeConvention=True,
                      orientation=orientation)
    foldedAntennaSkyMap = Amp**2*skyMapDF
    return foldedAntennaSkyMap

# integrate the folded sky map over the angles
def integrateOverAngles(foldedAntennaSkyMap):
    p = np.deg2rad(foldedAntennaSkyMap.index.values.astype(float))
    t = np.deg2rad(foldedAntennaSkyMap.columns.values.astype(float))
    T, P = np.meshgrid(t, p)
    angePrefactor=np.sin(T[:,::-1])
    integratedOverAngles=simps(simps(foldedAntennaSkyMap*angePrefactor,t),p)
    return integratedOverAngles

<p style='text-align: justify;'>
Ideally you should simulate as small time deltas as possible and than binned them approprietly.
For example, if the measured data are binned to 1 hour bins, calculating the simulated data at one hour steps might give you a 30 minutes time offset. Better is to simulate 15 minutes steps and binned them to 1 hour. This is more representative then. <br>
If the measured data set is binned to 15 minute bins, a simulated DataFrame with 15 minutes bins should be OK since the possible offset is only 7.5 minutes.
</p>

If the time binning is required, function <i>DFtimeBinning</i> can be used.

In [ ]:
# time binning
def DFtimeBinning(DF,timeBinWidth=1):
    new=[]
    newTimes=[]
    rowNumer, _ = DF.shape
    tstart=float(DF.index.values[0])
    tstep=float(DF.index.values[1])-float(DF.index.values[0])
    tWidth = int(timeBinWidth/tstep + 1) # how many cols have to be integrated to have the wanted frequency binning, the +1 is because the last  element is excluded
    totalNumberOfNewTimeRows=int((rowNumer-1)/(tWidth-1))
    for i in range(0,totalNumberOfNewTimeRows,1):
        newTimes.append(i*timeBinWidth)
        new.append(DF.iloc[i:i+tWidth,:].mean(axis=0))
    new = np.asarray(new)
    newTimes=np.asarray(newTimes)
    newDF = DFtemplateCreator(DF.columns.values, newTimes)
    newDF.iloc[:,:] = np.asarray(new)
    return newDF

***

## Power spectrum density

Assign the save folder for the results.

In [ ]:
saveFolder='./results/skySimulation/'

Further:<br>
- Set the path to the impedance files
- create antenna pattern classes for EW and NS orientation
- create sky temperature maps

In [ ]:
impedanceFile_EW = offlinePath+"share/auger-offline/doc/ExampleApplications/RDGalacticCalibrationTools/EW_impedance_RD.txt"
impedanceFile_NS = offlinePath+"share/auger-offline/doc/ExampleApplications/RDGalacticCalibrationTools/NS_impedance_RD.txt"

# antenna patter class
antennaEW = getAntennaPatternQuantities(antennaPathEW)
antennaNS = getAntennaPatternQuantities(antennaPathNS)
# for Polisensky's LFmap, these are by default generated in Celestial coordinated
#g_LFmap = LFmap(LFmapPath+"/healpyFits/")
# for lf map from pygdsem
#g_LFSS = LowFrequencySkyModel(freq_unit='MHz')

#use g_1 and g_2 instead

### Calculate the power spectrum density for the EW orientation with the maps from the 1st model.

In [ ]:
# model 1 EW
startFrequency=30.0
endFrequency=80.0
frequencySpacing=1.0
startLST=0
endLST=24
LSTspacing=1

powerSpectrumDensityDF_1_EW = simulateGalacticModulationCurve(antennaClass = antennaEW, skyMapClass = g_1, startFrequency = startFrequency,
                                                        endFrequency = endFrequency, frequencySpacing=frequencySpacing, coord=c_coord_1, 
                                                        orientation ='EW', returnOnlyIntegratedAngle=False,
                                                        startLST = startLST, endLST=endLST, LSTspacing=LSTspacing, impedanceFile=impedanceFile_EW)

In [ ]:
# for saving
powerSpectrumDensityDF_1_EW.to_csv(saveFolder+'powerSpectrumDensityDF_'+map_1_title+'_EW.csv')

In [ ]:
# for reading
powerSpectrumDensityDF_1_EW = pd.read_csv(saveFolder+'powerSpectrumDensityDF_'+map_1_title+'_EW.csv',index_col=0)

### Calculate the power spectrum density for the NS orientation with the maps from the 1st model.

In [ ]:
# model 1 NS
startFrequency=30.0
endFrequency=80.0
frequencySpacing=1.0
startLST=0
endLST=24
LSTspacing=1

powerSpectrumDensityDF_1_NS = simulateGalacticModulationCurve(antennaClass=antennaNS, skyMapClass=g_1, startFrequency=startFrequency,
                                                        endFrequency=endFrequency, frequencySpacing=frequencySpacing, coord=c_coord_1, 
                                                        orientation='NS', returnOnlyIntegratedAngle=False,
                                                        startLST=startLST, endLST=endLST, LSTspacing=LSTspacing, impedanceFile=impedanceFile_NS)

In [ ]:
# for saving
powerSpectrumDensityDF_1_NS.to_csv(saveFolder+'powerSpectrumDensityDF_'+map_1_title+'_NS.csv')

In [ ]:
# for reading
powerSpectrumDensityDF_1_NS = pd.read_csv(saveFolder+'powerSpectrumDensityDF_'+map_1_title+'_NS.csv',index_col=0)

### Calculate the power spectrum density for the EW orientation with the maps from the 2nd model.

In [ ]:
# map 2 EW
startFrequency=30.0
endFrequency=80.0
frequencySpacing=1.0
startLST=0
endLST=24
LSTspacing=1

powerSpectrumDensityDF_2_EW = simulateGalacticModulationCurve(antennaClass=antennaEW, skyMapClass=g_2, startFrequency=startFrequency,
                                                        endFrequency=endFrequency, frequencySpacing=frequencySpacing, coord = c_coord_2, 
                                                        orientation='EW', returnOnlyIntegratedAngle=False,
                                                        startLST=startLST, endLST=endLST, LSTspacing=LSTspacing, impedanceFile=impedanceFile_EW)

In [ ]:
# for saving
powerSpectrumDensityDF_2_EW.to_csv(saveFolder+'powerSpectrumDensityDF_'+map_2_title+'_EW.csv')

In [ ]:
# for reading
powerSpectrumDensityDF_LFmap_EW = pd.read_csv(saveFolder+'powerSpectrumDensityDF_'+map_2_title+'_EW.csv',index_col=0)

### Calculate the power spectrum density for the NS orientation with the maps from the 2nd model.

In [ ]:
# map 2 NS
startFrequency=30.0
endFrequency=80.0
frequencySpacing=1.0
startLST=0
endLST=24
LSTspacing=1

powerSpectrumDensityDF_2_NS = simulateGalacticModulationCurve(antennaClass=antennaNS, skyMapClass=g_2, startFrequency=startFrequency,
                                                        endFrequency=endFrequency, frequencySpacing=frequencySpacing, coord=c_coord_2, 
                                                        orientation='NS', returnOnlyIntegratedAngle=False,
                                                        startLST=startLST, endLST=endLST, LSTspacing=LSTspacing, impedanceFile=impedanceFile_NS)

In [ ]:
# for saving
powerSpectrumDensityDF_2_NS.to_csv(saveFolder+'powerSpectrumDensityDF_'+map_2_title+'_NS.csv')

In [ ]:
# for reading
powerSpectrumDensityDF_LFmap_NS = pd.read_csv(saveFolder+'powerSpectrumDensityDF_'+map_2_title+'_NS.csv',index_col=0)

### Plots - Power spectrum density in EW orientation

In [ ]:
orientation = 'EW'
#save=saveFolder
close=False
save=False

save1 = saveFolder+'powerSpectrumDensityDF_'+map_1_title+'_'+orientation
save2 = saveFolder+'powerSpectrumDensityDF_'+map_2_title+'_'+orientation
save3 = saveFolder+'Ratio_PowerSpectrumDensity_'+orientation

LST_grid, freq_grid = np.meshgrid(powerSpectrumDensityDF_1_EW.index.values.astype(float), powerSpectrumDensityDF_2_EW.columns.values.astype(float))

myPlots.plot3dnewV3(LST_grid, freq_grid, powerSpectrumDensityDF_1_EW.values.T*1e+18, slices=True, figureWidth=12, figureHeight=8,
                    xlabel='LST',ylabel='frequency [MHz]',cbarLabel='power spectrum density [pW/Hz]',xMajorLocator=2,yMajorLocator=10,
                    mainTitle=map_1_title+" - Power spectrum density in "+orientation, Cmin=0, Cmax=0.06,ymin=29.5,ymax=80.5,
                    save=save1,close=close)

myPlots.plot3dnewV3(LST_grid, freq_grid, powerSpectrumDensityDF_2_EW.values.T*1e+18, slices=True, figureWidth=12, figureHeight=8,
                    xlabel='LST',ylabel='frequency [MHz]',cbarLabel='power spectrum density [pW/Hz]',xMajorLocator=2,yMajorLocator=10,
                    mainTitle=map_2_title+" - Power spectrum density in "+orientation, Cmin=0, Cmax=0.06,ymin=29.5,ymax=80.5,
                    save=save2,close=close)

ratio = powerSpectrumDensityDF_1_EW.values/powerSpectrumDensityDF_2_EW.values
myPlots.plot3dnewV3(LST_grid, freq_grid, ratio.T,slices=True, figureWidth=12, figureHeight=8,
                    xlabel='LST',ylabel='frequency [MHz]',cbarLabel=r'$\frac{\mathrm{%s}}{\mathrm{%s}}$'%(map_1_title, map_2_title), xMajorLocator=2, yMajorLocator=10,
                    mainTitle="Ratio of "+map_1_title+" and "+map_2_title+" power spectrum densities in "+orientation, Cmin=0.9, Cmax=1.25,ymin=29.5,ymax=80.5, 
                    extend='both',
                    save=save3, close=close)

***
***


### Plots - Power spectrum density in NS orientation

In [ ]:
orientation = 'NS'
#save=saveFolder
close=False
save=False

save1 = saveFolder+'powerSpectrumDensityDF_'+map_1_title+'_NS'
save2 = saveFolder+'powerSpectrumDensityDF_'+map_2_title+'_NS'
save3 = saveFolder+'Ratio_PowerSpectrumDensity_NS'

LST_grid, freq_grid = np.meshgrid(powerSpectrumDensityDF_1_NS.index.values.astype(float), powerSpectrumDensityDF_1_NS.columns.values.astype(float))
myPlots.plot3dnewV3(LST_grid, freq_grid, powerSpectrumDensityDF_1_NS.values.T*1e+18, slices=True, figureWidth=12, figureHeight=8,
                    xlabel='LST',ylabel='frequency [MHz]',cbarLabel='power spectrum density [pW/Hz]',xMajorLocator=2,yMajorLocator=10,
                    mainTitle=map_1_title+" - Power spectrum density in "+orientation, Cmin=0, Cmax=0.06,ymin=29.5,ymax=80.5,
                    save=save1, close=close)

myPlots.plot3dnewV3(LST_grid, freq_grid, powerSpectrumDensityDF_2_NS.values.T*1e+18, slices=True, figureWidth=12, figureHeight=8,
                    xlabel='LST',ylabel='frequency [MHz]', cbarLabel='power spectrum density [pW/Hz]', xMajorLocator=2, yMajorLocator=10,
                    mainTitle=map_2_title+" - Power spectrum density in "+orientation, Cmin=0, Cmax=0.06, ymin=29.5, ymax=80.5,
                    save=save2, close=close)

ratio = powerSpectrumDensityDF_1_NS.values/powerSpectrumDensityDF_2_NS.values
myPlots.plot3dnewV3(LST_grid, freq_grid, ratio.T,slices=True, figureWidth=12, figureHeight=8,
                    xlabel='LST', ylabel='frequency [MHz]', cbarLabel=r'$\frac{\mathrm{%s}}{\mathrm{%s}}$'%(map_1_title, map_2_title), xMajorLocator=2, yMajorLocator=10,
                    mainTitle="Ratio of "+map_1_title+" and "+map_2_title+" power spectrum densities in "+orientation, Cmin=0.9, Cmax=1.25, ymin=29.5, ymax=80.5,
                    extend='both',
                    save=save3,close=close)

## Power
To get the power we need just to integrate the power spectrum density over the frequencies. <br>

 \begin{equation*}
P_{sky}(t,f) = \int_{f} \mathscr{P}_{sky}(t,f) df
\end{equation*} <br>

<p style='text-align: justify;'>
For Power in 1 Mhz bins a power spectrum density with at least 1 MHz spacing is needed. Ideally one should generate power spectrum density in the same frequencies as is the frequency delta of the measured data. For example, sampling frequency 250 MHz will produced spectrum with frequency delta 0.12 MHz, therefore 0.12 Mhz should be the step in the power spectrum density. However, the power spectrum density is not varying dramatically in 1 MHz steps, so 1 Mhz steps in power spectrum density should be fine.
</p>

Function <i>integratePowerSpectrumDensityDF2PowerSpectrum</i> will do the frequency integration.

In [ ]:
def integratePowerSpectrumDensityDF2PowerSpectrum(powerSpectrumDensityDF, fbinWidth=1):
    powerSpectrum=[]
    newFrequencies=[]
    _, colNumber = powerSpectrumDensityDF.shape
    fstart=float(powerSpectrumDensityDF.columns.values[0])
    fstep=float(powerSpectrumDensityDF.columns.values[1])-float(powerSpectrumDensityDF.columns.values[0])
    fwidth = int(fbinWidth/fstep + 1) # how many cols have to be integrated to have the wanted frequency binning, the +1 is because the last  element is excluded
    totalNumberOfNewFreqColumns=int((colNumber-1)/(fwidth-1))
    for i in range(0,totalNumberOfNewFreqColumns,1):
        newFrequencies.append(fstart+i*fbinWidth)
        #print(powerSpectrumDensityDF_LFSS.iloc[i:i+fwidth,:].index.values)
        powerSpectrum.append(simps(powerSpectrumDensityDF.iloc[:,i:i+fwidth].values, 
                                   powerSpectrumDensityDF.iloc[:,i:i+fwidth].columns.values.astype(float)*1e+6))
    powerSpectrum = np.asarray(powerSpectrum)
    newFrequencies=np.asarray(newFrequencies)
    powerSpectrumDF = DFtemplateCreator(newFrequencies,powerSpectrumDensityDF.index.values.astype(float))
    powerSpectrumDF.iloc[:,:] = np.asarray(powerSpectrum.T)
    return powerSpectrumDF

Integrate the calculated power spectrum densities to get the powers.

In [ ]:
powerDF_1_EW= integratePowerSpectrumDensityDF2PowerSpectrum(powerSpectrumDensityDF_1_EW)
powerDF_1_NS= integratePowerSpectrumDensityDF2PowerSpectrum(powerSpectrumDensityDF_1_NS)
powerDF_2_EW= integratePowerSpectrumDensityDF2PowerSpectrum(powerSpectrumDensityDF_2_EW)
powerDF_2_NS= integratePowerSpectrumDensityDF2PowerSpectrum(powerSpectrumDensityDF_2_NS)

In [ ]:
# for saving
powerDF_1_EW.to_csv(saveFolder+'powerDF_'+map_1_title+'_EW.csv')
# for saving
powerDF_1_NS.to_csv(saveFolder+'powerDF_'+map_1_title+'_NS.csv')
# for saving
powerDF_2_EW.to_csv(saveFolder+'powerDF_'+map_2_title+'_EW.csv')
# for saving
powerDF_2_NS.to_csv(saveFolder+'powerDF_'+map_2_title+'_NS.csv')

In [ ]:
# for reading
powerDF_1_EW = pd.read_csv(saveFolder+'powerDF_'+map_1_title+'_EW.csv',index_col=0)
# for reading
powerDF_1_NS = pd.read_csv(saveFolder+'powerDF_'+map_1_title+'_NS.csv',index_col=0)
# for reading
powerDF_2_EW = pd.read_csv(saveFolder+'powerDF_'+map_2_title+'_EW.csv',index_col=0)
# for reading
powerDF_2_NS = pd.read_csv(saveFolder+'powerDF_'+map_2_title+'_NS.csv',index_col=0)

### Plots - Power in EW orientation

In [ ]:
orientation = 'EW'
#save=saveFolder
close=False
save=False

save1 = saveFolder+'powerDF_'+map_1_title+'_EW'
save2 = saveFolder+'powerDF_'+map_2_title+'_EW'
save3 = saveFolder+'Ratio_Power_'+map_1_title+'_'+map_2_title+'_EW'

LST_grid, freq_grid = np.meshgrid(powerDF_1_EW.index.values.astype(float), powerDF_1_EW.columns.values.astype(float))
myPlots.plot3dnewV3(LST_grid, freq_grid, powerDF_1_EW.values.T*1e+12, slices=True,figureWidth=12, figureHeight=8,
                    xlabel='LST',ylabel='frequency [MHz]', cbarLabel='power [pW]',xMajorLocator=2, yMajorLocator=10,
                    mainTitle=map_1_title+" - power in "+orientation, Cmin=0, Cmax=0.06, ymin=29.5, ymax=80,
                    save=save1, close=close)

myPlots.plot3dnewV3(LST_grid, freq_grid, powerDF_2_EW.values.T*1e+12, slices=True, figureWidth=12, figureHeight=8,
                    xlabel='LST', ylabel='frequency [MHz]', cbarLabel='power [pW]', xMajorLocator=2, yMajorLocator=10,
                    mainTitle=map_2_title+" - power in "+orientation, Cmin=0, Cmax=0.06, ymin=29.5, ymax=80,
                    save=save2, close=close)

ratio = powerDF_1_EW.values/powerDF_2_EW.values
myPlots.plot3dnewV3(LST_grid, freq_grid, ratio.T, slices=True, figureWidth=12, figureHeight=8,
                    xlabel='LST', ylabel='frequency [MHz]', cbarLabel=r'$\frac{\mathrm{%s}}{\mathrm{%s}}$'%(map_1_title, map_2_title), xMajorLocator=2, yMajorLocator=10,
                    mainTitle="Ratio of "+map_1_title+" and "+map_2_title+" power spectrum densities in "+orientation, Cmin=0.9, Cmax=1.25, ymin=29.5, ymax=80,
                    extend='both',
                    save=save3, close=close)

### Plots - Power in NS orientation

In [ ]:
orientation = 'NS'
#save=saveFolder
close=False
save=False

save1 = saveFolder+'powerDF_'+map_1_title+'_NS'
save2 = saveFolder+'powerDF_'+map_2_title+'_NS'
save3 = saveFolder+'Ratio_Power_'+map_1_title+'_'+map_2_title+'_NS'

LST_grid, freq_grid = np.meshgrid(powerDF_1_NS.index.values.astype(float), powerDF_1_NS.columns.values.astype(float))
myPlots.plot3dnewV3(LST_grid, freq_grid, powerDF_1_NS.values.T*1e+12, slices=True, figureWidth=12, figureHeight=8,
                    xlabel='LST', ylabel='frequency [MHz]', cbarLabel='power [pW]', xMajorLocator=2, yMajorLocator=10,
                    mainTitle=map_1_title+" - power in "+orientation, Cmin=0, Cmax=0.06, ymin=29.5, ymax=80,
                    save=save1, close=close)

myPlots.plot3dnewV3(LST_grid, freq_grid, powerDF_2_NS.values.T*1e+12, slices=True, figureWidth=12, figureHeight=8,
                    xlabel='LST', ylabel='frequency [MHz]', cbarLabel='power [pW]', xMajorLocator=2, yMajorLocator=10,
                    mainTitle=map_2_title+" - power in "+orientation, Cmin=0, Cmax=0.06, ymin=29.5, ymax=80,
                    save=save2, close=close)

ratio = powerDF_1_NS.values/powerDF_2_NS.values
myPlots.plot3dnewV3(LST_grid, freq_grid, ratio.T, slices=True, figureWidth=12, figureHeight=8,
                    xlabel='LST', ylabel='frequency [MHz]', cbarLabel=r'$\frac{\mathrm{%s}}{\mathrm{%s}}$'%(map_1_title, map_2_title), xMajorLocator=2, yMajorLocator=10,
                    mainTitle="Ratio of "+map_1_title+" and "+map_2_title+" power spectrum densities in "+orientation, Cmin=0.9, Cmax=1.25, ymin=29.5, ymax=80,
                    extend='both',
                    save=save3, close=close)